In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings('ignore')  #Suppress filter warnings

# 1. Reading Data

In [2]:
chillerdata = pd.read_excel('230720_HSC_CleanedData.xlsx',sheet_name='CH1')

chillerdata.drop(columns='Unnamed: 0',axis=0,inplace=True)
chiller_capacity = 600
#Convert the load to kW
chillerdata['Load'] = chillerdata['Load']*3.5169

chillerdata.head()

,Time_range,Day_of_week,Occupancy,month,Tdb,Twb,CHLR_LCHWT,CHLR_LCWT,CHLR_RCHWT,CHLR_RCWT,...,CW_flow,CWET,CWLT,ELE,CHLR_ON,PLR,PLR_sec,chillers_no,Load,kW_ton
0,2019-01-01 00:00:00,2,Unoccupied,Jan,46.940,44.930620,44.950,54.825,48.775,56.575,...,1798.325,54.907,56.215,31.00,1,0.155743,1.383711,1,328.638644,0.331744
1,2019-01-01 01:00:00,2,Unoccupied,Jan,46.604,44.656086,44.700,54.650,48.650,56.375,...,1801.663,54.748,56.078,30.75,1,0.160112,1.417633,1,337.859058,0.320088
2,2019-01-01 02:00:00,2,Unoccupied,Jan,44.060,42.622744,44.900,54.875,48.550,56.500,...,1802.898,54.930,56.192,31.25,1,0.154403,1.356249,1,325.811111,0.337322
3,2019-01-01 03:00:00,2,Unoccupied,Jan,43.934,43.015251,44.975,54.875,48.700,56.450,...,1801.423,54.867,56.120,31.25,1,0.155532,1.421435,1,328.193628,0.334873
4,2019-01-01 04:00:00,2,Unoccupied,Jan,42.875,42.421400,45.000,54.925,48.600,56.650,...,1800.285,54.973,56.280,30.50,1,0.152975,1.341871,1,322.798667,0.332298


**Weather Data**

In [3]:
weather = pd.read_csv('NOAA__747460__03904 - EASTERWOOD FIELD AIRPORT - 01012019 to 01012021.csv')
weather.columns = ['Time_range','Tdb','Twb','Tdp']

weather['Time_range'] = pd.to_datetime(weather['Time_range'])
weather['year'] = weather['Time_range'].dt.year

#Weather conditions to be used to determine the parameter, psi
Twb_max = weather[weather['year']==2019]['Twb'].max()
Twb_min = weather[weather['year']==2019]['Twb'].min()

FileNotFoundError: [Errno 2] No such file or directory: 'NOAA__747460__03904 - EASTERWOOD FIELD AIRPORT - 01012019 to 01012021.csv'

# 2. Monthly and consecutive metering period analysis 

**1 Month samples**
* Sample 1 = Jan
* Sample 2 = Feb
* ...
* Sample 11 = Nov
* Sample 12 = Dec


**2 months period samples**
* Sample 13 = Jan & Feb
* Sample 14 = Feb & Mar
* Sample 15 = Mar & Apr
* ...
* Sample 23 = Nov & Dec
* Sample 24 = Dec & Jan


**3 months period samples**
* Sample 25 = Jan, Feb & Mar
* Sample 26 = Feb, Mar & Apr
* Sample 27 = Mar, Apr & Jun
* ...
* Sample 36 = Dec, Jan & Feb


**4 months period samples**
* Sample 37 = Jan, Feb, Mar & Apr
* Sample 38 = Feb, Mar, Apr, May
* Sample 39 = Mar, Apr, May, Jun
* ...
* Sample 47 = Nov, Dec, Jan & Feb
* Sample 48 = Dec, Jan, Feb, & Mar

**NOTE**
* 50% of the data of each month is used as sample (metering) data.

# 3. Creation of the sampling and test datasets

In [ ]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
month_samples = ['Jan_sample','Feb_sample','Mar_sample','Apr_sample','May_sample','Jun_sample',
                 'Jul_sample','Aug_sample','Sep_sample','Oct_sample','Nov_sample','Dec_sample']

#Dictionary of datasets per month
months_data = {}
for month in months:
    months_data[month] = chillerdata[chillerdata['month']==month]

#Dictionary of data samples per month
sample_size = 0.5  #Sample size = 50%
seed_number = 42
month_samples_data = {}
for month_sample in month_samples:
    month_samples_data[month_sample] = months_data[month_sample[:3]].sample(frac=sample_size, random_state=seed_number)
    
#Metering sample combinations
metering_sample_sets = {'1_month':{},'2_months':{},'3_months':{},'4_months':{}}

#1 month metering data
metering_sample_sets['1_month']['sample1'] = month_samples_data['Jan_sample']   
metering_sample_sets['1_month']['sample2'] = month_samples_data['Feb_sample']
metering_sample_sets['1_month']['sample3'] = month_samples_data['Mar_sample']
metering_sample_sets['1_month']['sample4'] = month_samples_data['Apr_sample']
metering_sample_sets['1_month']['sample5'] = month_samples_data['May_sample']
metering_sample_sets['1_month']['sample6'] = month_samples_data['Jun_sample']
metering_sample_sets['1_month']['sample7'] = month_samples_data['Jul_sample']
metering_sample_sets['1_month']['sample8'] = month_samples_data['Aug_sample']
metering_sample_sets['1_month']['sample9'] = month_samples_data['Sep_sample']
metering_sample_sets['1_month']['sample10'] = month_samples_data['Oct_sample']
metering_sample_sets['1_month']['sample11'] = month_samples_data['Nov_sample']
metering_sample_sets['1_month']['sample12'] = month_samples_data['Dec_sample']


#2 months metering data
metering_sample_sets['2_months']['sample1'] = pd.concat([month_samples_data['Jan_sample'],month_samples_data['Feb_sample']])
metering_sample_sets['2_months']['sample2'] = pd.concat([month_samples_data['Feb_sample'],month_samples_data['Mar_sample']])
metering_sample_sets['2_months']['sample3'] = pd.concat([month_samples_data['Mar_sample'],month_samples_data['Apr_sample']])
metering_sample_sets['2_months']['sample4'] = pd.concat([month_samples_data['Apr_sample'],month_samples_data['May_sample']])
metering_sample_sets['2_months']['sample5'] = pd.concat([month_samples_data['May_sample'],month_samples_data['Jun_sample']])
metering_sample_sets['2_months']['sample6'] = pd.concat([month_samples_data['Jun_sample'],month_samples_data['Jul_sample']])
metering_sample_sets['2_months']['sample7'] = pd.concat([month_samples_data['Jul_sample'],month_samples_data['Aug_sample']])
metering_sample_sets['2_months']['sample8'] = pd.concat([month_samples_data['Aug_sample'],month_samples_data['Sep_sample']])
metering_sample_sets['2_months']['sample9'] = pd.concat([month_samples_data['Sep_sample'],month_samples_data['Oct_sample']])
metering_sample_sets['2_months']['sample10'] = pd.concat([month_samples_data['Oct_sample'],month_samples_data['Nov_sample']])
metering_sample_sets['2_months']['sample11'] = pd.concat([month_samples_data['Nov_sample'],month_samples_data['Dec_sample']])
metering_sample_sets['2_months']['sample12'] = pd.concat([month_samples_data['Dec_sample'],month_samples_data['Jan_sample']])

# 3 months metering data
metering_sample_sets['3_months']['sample1'] = pd.concat([month_samples_data['Jan_sample'],month_samples_data['Feb_sample'],
                                                     month_samples_data['Mar_sample']])
metering_sample_sets['3_months']['sample2'] = pd.concat([month_samples_data['Feb_sample'],month_samples_data['Mar_sample'],
                                                    month_samples_data['Apr_sample']])
metering_sample_sets['3_months']['sample3'] = pd.concat([month_samples_data['Mar_sample'],month_samples_data['Apr_sample'],
                                                     month_samples_data['May_sample']])
metering_sample_sets['3_months']['sample4'] = pd.concat([month_samples_data['Apr_sample'],month_samples_data['May_sample'],
                                                     month_samples_data['Jun_sample']])
metering_sample_sets['3_months']['sample5'] = pd.concat([month_samples_data['May_sample'],month_samples_data['Jun_sample'],
                                                     month_samples_data['Jul_sample']])
metering_sample_sets['3_months']['sample6'] = pd.concat([month_samples_data['Jun_sample'],month_samples_data['Jul_sample'],
                                                     month_samples_data['Aug_sample']])
metering_sample_sets['3_months']['sample7'] = pd.concat([month_samples_data['Jul_sample'],month_samples_data['Aug_sample'],
                                                     month_samples_data['Sep_sample']])
metering_sample_sets['3_months']['sample8'] = pd.concat([month_samples_data['Aug_sample'],month_samples_data['Sep_sample'],
                                                     month_samples_data['Oct_sample']])
metering_sample_sets['3_months']['sample9'] = pd.concat([month_samples_data['Sep_sample'],month_samples_data['Oct_sample'],
                                                     month_samples_data['Nov_sample']])
metering_sample_sets['3_months']['sample10'] = pd.concat([month_samples_data['Oct_sample'],month_samples_data['Nov_sample'],
                                                      month_samples_data['Dec_sample']])
metering_sample_sets['3_months']['sample11'] = pd.concat([month_samples_data['Nov_sample'],month_samples_data['Dec_sample'],
                                                      month_samples_data['Jan_sample']])
metering_sample_sets['3_months']['sample12'] = pd.concat([month_samples_data['Dec_sample'],month_samples_data['Jan_sample'],
                                                      month_samples_data['Feb_sample']])


# 4 months metering data
metering_sample_sets['4_months']['sample1'] = pd.concat([month_samples_data['Jan_sample'],month_samples_data['Feb_sample'],
                                                     month_samples_data['Mar_sample'],month_samples_data['Apr_sample']])
metering_sample_sets['4_months']['sample2'] = pd.concat([month_samples_data['Feb_sample'],month_samples_data['Mar_sample'],
                                                    month_samples_data['Apr_sample'],month_samples_data['May_sample']])
metering_sample_sets['4_months']['sample3'] = pd.concat([month_samples_data['Mar_sample'],month_samples_data['Apr_sample'],
                                                     month_samples_data['May_sample'],month_samples_data['Jun_sample']])
metering_sample_sets['4_months']['sample4'] = pd.concat([month_samples_data['Apr_sample'],month_samples_data['May_sample'],
                                                     month_samples_data['Jun_sample'],month_samples_data['Jul_sample']])
metering_sample_sets['4_months']['sample5'] = pd.concat([month_samples_data['May_sample'],month_samples_data['Jun_sample'],
                                                     month_samples_data['Jul_sample'],month_samples_data['Aug_sample']])
metering_sample_sets['4_months']['sample6'] = pd.concat([month_samples_data['Jun_sample'],month_samples_data['Jul_sample'],
                                                     month_samples_data['Aug_sample'],month_samples_data['Sep_sample']])
metering_sample_sets['4_months']['sample7'] = pd.concat([month_samples_data['Jul_sample'],month_samples_data['Aug_sample'],
                                                     month_samples_data['Sep_sample'],month_samples_data['Oct_sample']])
metering_sample_sets['4_months']['sample8'] = pd.concat([month_samples_data['Aug_sample'],month_samples_data['Sep_sample'],
                                                     month_samples_data['Oct_sample'],month_samples_data['Nov_sample']])
metering_sample_sets['4_months']['sample9'] = pd.concat([month_samples_data['Sep_sample'],month_samples_data['Oct_sample'],
                                                     month_samples_data['Nov_sample'],month_samples_data['Dec_sample']])
metering_sample_sets['4_months']['sample10'] = pd.concat([month_samples_data['Oct_sample'],month_samples_data['Nov_sample'],
                                                      month_samples_data['Dec_sample'],month_samples_data['Jan_sample']])
metering_sample_sets['4_months']['sample11'] = pd.concat([month_samples_data['Nov_sample'],month_samples_data['Dec_sample'],
                                                      month_samples_data['Jan_sample'],month_samples_data['Feb_sample']])
metering_sample_sets['4_months']['sample12'] = pd.concat([month_samples_data['Dec_sample'],month_samples_data['Jan_sample'],
                                                      month_samples_data['Feb_sample'],month_samples_data['Mar_sample']])

#Sample names
samples1 = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
samples2 = ['Jan-Feb','Feb-Mar','Mar-Apr','Apr-May','May-Jun','Jun-Jul','Jul-Aug','Aug-Sep','Sep-Oct',
            'Oct-Nov','Nov-Dec','Dec-Jan']
samples3 = ['Jan-Mar','Feb-Apr','Mar-May','Apr-Jun','May-Jul','Jun-Aug','Jul-Sep','Aug-Oct','Sep-Nov',
            'Oct-Dec','Nov-Jan','Dec-Feb']
samples4 = ['Jan-Apr','Feb-May','Mar-Jun','Apr-Jul','May-Aug','Jun-Sep','Jul-Oct','Aug-Nov','Sep-Dec',
            'Oct-Jan','Nov-Feb','Dec-Mar']

samples = {'1_month':samples1,'2_months':samples2,'3_months':samples3,'4_months':samples4}

#Test datasets fo each sample
test_sample_sets = {'1_month':{},'2_months':{},'3_months':{},'4_months':{}}

for sample_set in test_sample_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        chiller_copy = chillerdata.copy()
        cond = chiller_copy['Time_range'].isin(metering_sample_sets[sample_set][sample]['Time_range'])
        test_sample_sets[sample_set][sample] = chiller_copy.drop(chiller_copy[cond].index)

metering_sample_sets['1_month']['sample6']

# 4. Classes and functions

**4.1 Model Power Class**

In [ ]:
class ModelPower():
    def __init__(self,vars_df):
        """vars_df: The dataframe to which to store the predicted electrical consumption"""
        self.vars_df = vars_df

    def temp_indep_model(Load,y_pred):
        """Parameters:
        Load: The chiller load in kW
        y_pred: The model's predicted variable, input as a series
        """
        ELE_pred = Load*y_pred
        return ELE_pred
    
    def quasi_empirical_model(self,Load,y_pred,CWRT,CHWST):
        """Parameters:/n
        Load: The chiller load in kW, supplied as a series from the df of either the training or testing dataset
        y_pred: The model's predicted variable, input as a series from the df of the training dataset
        CWRT: The condenser water return temperature in K. As a series extracted from the df of the training or testing dataset
        CHWST: Chilled water supply temperature in K. As a series extracted from the df of the training or testing dataset
        """
        COP_rec = (y_pred+Load*CWRT/CHWST)/Load-1
        self.vars_df['ELE_pred'] = Load*COP_rec
        return self.vars_df['ELE_pred']
    
    def GN_fundamental_model(self,Load,y_pred,CWRT,CHWST):
        """
        This function can be used with the Fundamental GN model with and without intercept, and with split Qleak term
        It can also be used with the Fundamental GN model with variable DS term
        Parameters:
        Load: The chiller load in kW, supplied as a series from the df of the training or testing datasets
        y_pred: The model's predicted variable, input as a series from the df of the metering sample/n
        CWRT: The condenser water return temperature in K. As a series extracted from the df of the metering sample/n
        CHWST: Chilled water supply temperature in K. As a series extracted from the df of the metering sample/n
        """
        COP_rec = (y_pred+1)*CWRT/CHWST-1
        self.vars_df['ELE_pred'] =Load*COP_rec
        return self.vars_df['ELE_pred']
    
    def GN_Foliaco_model(self,Load,y_pred,CWRT,CHWST):
        """
        Parameters:
        Load: The chiller load in kW, supplied as a series from the df of the training or testing datasets
        y_pred: The model's predicted variable, input as a series from the df of the metering sample/n
        CWRT: The condenser water return temperature in K. As a series extracted from the df of the metering sample/n
        CHWST: Chilled water supply temperature in K. As a series extracted from the df of the metering sample/n
        """
        self.vars_df['ELE_pred'] = y_pred+(CWRT-CHWST)*Load/CHWST
        return self.vars_df['ELE_pred']
    
    def biquadratic_model(self,Load,y_pred):
        """
        Parameters:
        Load: The chiller load in kW, supplied as a series from the df of the training or testing datasets
        y_pred: The model's predicted variable, input as a series from the df of the training or testing datasets
        """
        self.vars_df['ELE_pred'] = Load*y_pred
        return self.vars_df['ELE_pred']
    
    def multivariate_model(self,Load,y_pred):
        """
        Parameters:/n
        Load: The chiller load in kW, supplied as a series from the df of the metering sample/n
        y_pred: The model's predicted variable, input as a series from the df of the metering sample/n
        """
        self.vars_df['ELE_pred'] = Load/y_pred
        return self.vars_df['ELE_pred']

**4.2 Feature generator class**

In [ ]:
class FeatureGenerator:
        
    class TempIndep:
        def __init__(self,vars_df):
            """
            The df vars_df is entered with columns {'Load','ELE'}, Load in kW and ELE in kW
            """
            self.vars_df = vars_df
        def y_label(self):
            COP_rec = self.vars_df['ELE']/(self.vars_df['Load'])
            return COP_rec 
        def X_features(self):
            Load_rec = 1/(self.vars_df['Load'])
            return Load_rec.to_numpy().reshape((-1,1))
    
    class QuasiEmpirical:
        def __init__(self,vars_df):
            """
            The df vars_df is entered with columns {'CWRT','CHWST','Load','ELE'} 
            The temperarures are in F and load in kW
            The symbols of the variables should be used as is
            """
            self.vars_df = vars_df
        def y_label(self):
            self.vars_df['COP'] = self.vars_df['Load']/self.vars_df['ELE']
            #Convert the temps to K
            self.vars_df['CWRT'] = (self.vars_df['CWRT']-32)*5/9+273.15
            self.vars_df['CHWST'] = (self.vars_df['CHWST']-32)*5/9+273.15 
            self.vars_df['y'] = (1+1/self.vars_df['COP'])*self.vars_df['Load']-self.vars_df['Load']*self.vars_df['CWRT']/self.vars_df['CHWST']
            return self.vars_df['y']
        def X_features(self):
            self.vars_df['x1'] = self.vars_df['CWRT']
            self.vars_df['x2'] = self.vars_df['CWRT']/self.vars_df['CHWST']
            return self.vars_df[['x1','x2']]
    class GNFundamental:
        def __init__(self,vars_df):
            """
            This works well for GN fundamental model, with or without intercept
            The df vars_df is entered with columns {'CWRT','CHWST','Load','ELE'}
            The temperarures are in F and Load and ELE in kW
            The symbols of the variables should be used as is
            """
            self.vars_df = vars_df
            self.vars_df['COP'] = self.vars_df['Load']/self.vars_df['ELE']
        def y_label(self):
            #Convert the temps to K
            self.vars_df['CWRT'] = (self.vars_df['CWRT']-32)*5/9+273.15
            self.vars_df['CHWST'] = (self.vars_df['CHWST']-32)*5/9+273.15 
            self.vars_df['y'] = (1+1/self.vars_df['COP'])*self.vars_df['CHWST']/self.vars_df['CWRT']-1
            return self.vars_df['y']
        def X_features(self):
            #GN model regression variables
            self.vars_df['x1'] = self.vars_df['CHWST']/self.vars_df['Load']
            self.vars_df['x2'] = (self.vars_df['CWRT']-self.vars_df['CHWST'])/(self.vars_df['CWRT']*self.vars_df['Load'])
            self.vars_df['x3'] = self.vars_df['Load']/self.vars_df['CWRT']*(1+1/self.vars_df['COP'])
            return self.vars_df[['x1','x2','x3']]
        
    class GNFundamental_DS:
        def __init__(self,vars_df):
            """
            The df vars_df is entered with columns {'CWRT','CHWST','Load','ELE'}
            The temperarures are in F and Load and ELE in kW
            The symbols of the variables should be used as is
            """
            self.vars_df = vars_df
            self.vars_df['COP'] = self.vars_df['Load']/self.vars_df['ELE']
        def y_label(self):
            #Convert the temps to K
            self.vars_df['CWRT'] = (self.vars_df['CWRT']-32)*5/9+273.15
            self.vars_df['CHWST'] = (self.vars_df['CHWST']-32)*5/9+273.15 
            self.vars_df['y'] = (1+1/self.vars_df['COP'])*self.vars_df['CHWST']/self.vars_df['CWRT']-1
            return self.vars_df['y']
        def X_features(self,Tcho_des = (44-32)*5/9+273.15,Q_rated = 2110):
            self.vars_df['x1'] = self.vars_df['CHWST']/self.vars_df['Load']
            #Rated parameters
            self.vars_df['x2'] = self.vars_df['x1']*self.vars_df['CHWST']/Tcho_des
            self.vars_df['x3'] = self.vars_df['x1']*self.vars_df['Load']/Q_rated
            self.vars_df['x4'] = (self.vars_df['CWRT']-self.vars_df['CHWST'])/(self.vars_df['CWRT']*self.vars_df['Load'])
            self.vars_df['x5'] = self.vars_df['Load']/self.vars_df['CWRT']*(1+1/self.vars_df['COP'])
            return self.vars_df[['x1','x2','x3','x4','x5']]
        
    class GNSplit_Qleak:
        def __init__(self,vars_df):
            """
            The df vars_df is entered with columns {'CWRT','CHWST','Load','ELE'}
            The temperarures are in F and Load and ELE in kW
            The symbols of the variables should be used as is
            """
            self.vars_df = vars_df
            self.vars_df['COP'] = self.vars_df['Load']/self.vars_df['ELE']
        def y_label(self):
            #Convert the temps to K
            self.vars_df['CWRT'] = (self.vars_df['CWRT']-32)*5/9+273.15
            self.vars_df['CHWST'] = (self.vars_df['CHWST']-32)*5/9+273.15
            #Model y label
            self.vars_df['y'] = (1+1/self.vars_df['COP'])*self.vars_df['CHWST']/self.vars_df['CWRT']-1
            return self.vars_df['y']
        def X_features(self):
            self.vars_df['x1'] = self.vars_df['CHWST']/self.vars_df['Load']
            self.vars_df['x2'] = (self.vars_df['CWRT']-self.vars_df['CHWST'])/(self.vars_df['CWRT']*self.vars_df['Load'])
            self.vars_df['x3'] = self.vars_df['CHWST']/(self.vars_df['Load']*self.vars_df['CWRT'])
            self.vars_df['x4'] = self.vars_df['Load']/self.vars_df['CWRT']*(1+1/self.vars_df['COP'])
            return self.vars_df[['x1','x2','x3','x4']]
    class GNFoliaco:
        def __init__(self,vars_df):
            """
            The df vars_df is entered with columns {'CWRT','CHWST','Load','ELE'}
            The temperarures are in F and Load and ELE in kW
            The symbols of the variables should be used as is
            """
            self.vars_df = vars_df
        def y_label(self):
            #Convert the temps to K
            self.vars_df['CWRT'] = (self.vars_df['CWRT']-32)*5/9+273.15
            self.vars_df['CHWST'] = (self.vars_df['CHWST']-32)*5/9+273.15
            #Model y label
            self.vars_df['y'] = self.vars_df['ELE']-(self.vars_df['CWRT']-
                                                     self.vars_df['CHWST'])*self.vars_df['Load']/self.vars_df['CHWST']
            return self.vars_df['y']
        def X_features(self):
            self.vars_df['x1'] = self.vars_df['CWRT']
            self.vars_df['x2'] = (self.vars_df['CWRT']-self.vars_df['CHWST'])/self.vars_df['CHWST']
            self.vars_df['x3'] = (self.vars_df['Load']**2+self.vars_df['Load']*self.vars_df['ELE'])/self.vars_df['CHWST']
            return self.vars_df[['x1','x2','x3']]
        
    class BiQuadratic:
        def __init__(self,vars_df):
            """
            The df vars_df is entered with columns {'CWRT','Load','ELE'}
            The temperarures are in F and Load and ELE in kW
            The symbols of the variables should be used as is
            """
            self.vars_df = vars_df
            self.vars_df['COP'] = self.vars_df['Load']/self.vars_df['ELE']
        def y_label(self):
            self.vars_df['y'] = 1/self.vars_df['COP']
            return self.vars_df['y']
        def X_features(self):
            #Convert the temps to C
            self.vars_df['CWRT'] = (self.vars_df['CWRT']-32)*5/9
            #Features
            self.vars_df['x1'] = 1/self.vars_df['Load']
            self.vars_df['x2'] = self.vars_df['Load']
            self.vars_df['x3'] = self.vars_df['CWRT']/self.vars_df['Load']
            self.vars_df['x4'] = self.vars_df['CWRT']**2/self.vars_df['Load']
            self.vars_df['x5'] = self.vars_df['CWRT']
            self.vars_df['x6'] = self.vars_df['CWRT']*self.vars_df['Load']
            self.vars_df['x7'] = self.vars_df['CWRT']**2
            self.vars_df['x8'] = self.vars_df['Load']*self.vars_df['CWRT']**2
            return self.vars_df[['x1','x2','x3','x4','x5','x6','x7','x8']]
    class MultiVariate:
        def __init__(self,vars_df):
            """
            The df vars_df is entered with columns {'CWRT','CHWRT','Load','ELE'}
            The temperarures are in F and Load and ELE in kW
            The symbols of the variables should be used as is
            """
            self.vars_df = vars_df
            self.vars_df['COP'] = self.vars_df['Load']/self.vars_df['ELE']
        def y_label(self):
            self.vars_df['y'] =self.vars_df['COP']
            return self.vars_df['y']
        def X_features(self):
            #Convert the temps to C
            self.vars_df['CWRT'] = (self.vars_df['CWRT']-32)*5/9
            self.vars_df['CHWRT'] = (self.vars_df['CHWRT']-32)*5/9
            #Features
            polynomial_converter = PolynomialFeatures(degree=2,include_bias=False)
            poly_features = polynomial_converter.fit_transform(self.vars_df[['CWRT','CHWRT','Load']])
            X_features = pd.DataFrame(data=poly_features,index=self.vars_df.index,
                                      columns=['x1','x2','x3','x4','x5','x6','x7','x8','x9'])
            return X_features

**4.3 Linear regression class**

In [ ]:
class Regression():
    """
    This function creates a linear (single or several variables) regression with or without intercept/n
    Parameters:
    X_train: A df of X features for a given model
    y_train: A series of the y label of the model
    """
    def __init__(self,X_train,y_train):
        self.X_train = X_train
        self.y_train = y_train
    def no_intercept(self):
        model = LinearRegression(fit_intercept=False)
        model.fit(self.X_train,self.y_train)
        return model
    def with_intercept(self):
        model = LinearRegression(fit_intercept=True)
        model.fit(self.X_train,self.y_train)
        return model

**4.4 Predictions and stats class**

In [ ]:
class ypred_and_stats():
        
    def y_pred(model,X):
        """
        Model = Originally created regression model
        X = Dataframe of X features, created from the FeatureGenerator Class
        """
        y_pred = model.predict(X)
        return y_pred
    def pred_stats(y,y_pred):
        """
        This function returns a dictionary of prediction stats {R2,RMSE,CV}, in that order
        
        y = y-label extracted from the FeatureGenerator class functions or measured ELE; entered as a Series 
        y_pred = Model-predicted y-label from the y_pred function of the ypred_and_stats class or predicted ELE; entered as a Series
        
        """
        MAE = mean_absolute_error(y,y_pred)
        MSE = mean_squared_error(y,y_pred)
        RMSE = np.sqrt(MSE)
        CV = RMSE*100/np.mean(y)
        SE = (y_pred - y)**2
        ST = (y - y.mean())**2
        R2 = 1-SE.sum()/ST.sum()
        
        stats = {'R2':R2,'RMSE':RMSE,'CV':CV}
        return stats

**4.5 Model train data performance function**

In [ ]:
def train_regression_stats(model,model_vars,X_train,y_train):
    #Model Parameters and statistics
    interc = model.intercept_
    R2 = model.score(X_train,y_train)
    
    MAE_ytrain = mean_absolute_error(y_train,model_vars['train_pred'])
    MAE_perc = mean_absolute_percentage_error(y_train,model_vars['train_pred'])
    MBE_ytrain = MAE_ytrain*100/np.mean(y_train)
    MSE_ytrain = mean_squared_error(y_train,model_vars['train_pred'])
    RMSE_ytrain = np.sqrt(MSE_ytrain)
    CV_ytrain = RMSE_ytrain*100/np.mean(y_train)
    train_reg_stats = [MBE_ytrain,RMSE_ytrain,CV_ytrain]
    return train_reg_stats, model_vars  
    
def train_power_prediction_stats(model_vars):
    #POWER PREDICTION STATS: TRAIN DATA
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])
    train_power_stats = [MBE_kWtrain,RMSE_kWtrain,CV_kWtrain]
    return train_power_stats

**4.6 Model test data performance function**

In [ ]:
def test_regression_stats(model,test_vars,X_test,y_test):
    #STATS
    MAE_ytest = mean_absolute_error(y_test,test_vars['test_pred'])
    MBE_ytest = MAE_ytest*100/np.mean(y_test)
    MSE_ytest = mean_squared_error(y_test,test_vars['test_pred'])
    RMSE_ytest = np.sqrt(MSE_ytest)
    CV_ytest = RMSE_ytest*100/np.mean(y_test)
    test_reg_stats = [MBE_ytest,RMSE_ytest,CV_ytest]
    return test_reg_stats,test_vars

def test_power_prediction_stats(test_vars):
    #POWER PREDICTION STATS: TEST DATA
    MAE_kWtest = mean_absolute_error(test_vars['ELE'],test_vars['ELE_pred'])
    MBE_kWtest = MAE_kWtest*100/np.mean(test_vars['ELE'])
    MSE_kWtest = mean_squared_error(test_vars['ELE'],test_vars['ELE_pred'])
    RMSE_kWtest = np.sqrt(MSE_kWtest)
    CV_kWtest = RMSE_kWtest*100/np.mean(test_vars['ELE'])
    test_power_stats = [MBE_kWtest,RMSE_kWtest,CV_kWtest]
    return test_power_stats

**4.7 Function to combine the performance stats into a dataframe**

In [ ]:
def stats_combiner(stats_list):
    combined_stats_df = pd.DataFrame(stats_list.reshape((12,12)), index=samples[sample_set], 
                              columns=['MBE_ytrain','RMSE_ytrain','CV_ytrain','MBE_kWtrain','RMSE_kWtrain',
                                       'CV_kWtrain','MBE_ytest','RMSE_ytest','CV_ytest',
                                       'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    combined_stats_df['CV_ratio'] = combined_stats_df['CV_ytest']/combined_stats_df['CV_ytrain']
    combined_stats_df['CV_kW_ratio'] = combined_stats_df['CV_kWtest']/combined_stats_df['CV_kWtrain']
    combined_stats_df['CV_kW_net'] = 0.5*((combined_stats_df['CV_kWtest']+combined_stats_df['CV_kWtrain'])/2+
                                      np.sqrt(combined_stats_df['CV_kWtest']*combined_stats_df['CV_kWtrain']))*(combined_stats_df['CV_kW_ratio'])**0.25
    
    return combined_stats_df

**4.8 Function to generate the CVratio data for plotting**

In [ ]:
def CVratio_generator(combined_stats_sets):
    CVratio_data = pd.DataFrame({'RMSE_kWtrain':[],'RMSE_kWtest':[],'CV_kWtrain':[],'CV_kWtest':[],
                                 'CV_ratio':[],'CV_kW_ratio':[]})
    for sample_set in combined_stats_sets.keys():
        CVratio_data = pd.concat([CVratio_data,
                                  combined_stats_sets[sample_set][['RMSE_kWtrain','RMSE_kWtest',
                                                                   'CV_kWtrain','CV_kWtest','CV_ratio',
                                                                   'CV_kW_ratio','CV_kW_net']]])
    
    CVratio_data['Sample_set'] = ['1_month']*12+['2_months']*12+['3_months']*12+['4_months']*12

    #Creating a new column containing the index values
    CVratio_data = CVratio_data.reset_index().rename(columns={'index': 'metering period'})
    return CVratio_data

**4.9 CVratio versus $\psi$ plot function --> Nolonger using this one!**

In [ ]:
def psi_plot(CVratio_data,legend_font_size=8):
    plt.figure(figsize=(6,5),dpi=100)

    ax = sns.scatterplot(data=CVratio_data,x='DT_dbwb/Tdbwb_range',y='CV_kW_ratio',marker = 'o',hue='Sample_set');
    ax.set_ylabel(r'$\frac{CV_{test}}{CV_{train}}$')
    ax.set_xlabel(r'$\psi$')
    ax.set_ylim([0.0,12]);  #This to be changed according to the model predictions
    ax.set_xlim([0.0,1]);

    #Add a secondary axis
    ax2 = ax.twinx()
    sns.scatterplot(data=CVratio_data,x='DT_dbwb/Tdbwb_range',y='CV_kWtrain',marker = '^',ax = ax2,hue='Sample_set');
    ax2.set_ylabel(r'$CV_{kW,train}$')
    ax2.set_ylim([0,30])

    plots = [ax.get_children()[0], ax2.get_children()[0]]
    labels = ["kW CV ratio","kW CV"]
    plt.legend(plots, labels, loc="upper left",fontsize=legend_font_size)

**4.10 CV barplots function**

In [ ]:
def CV_barplots(CVratio_data,ylim,figtitle,text_x=3,text_y=1,title_x=5,title_y=-12):
    """This function creates CV barplots for 1-month, 2-month, 3-month and 4-month metering samples\n.
    
    Parameters:
    CVratio_data: A dataframe containing the data to plot.\n
    ylim: y-axes limits entered as a list, [ymin,ymax]\n
    figtitle: Figure title, entered as a string
    text_x = Horizontal position of the 'Typical M&V CV Requirement' text on the plot
    text_y = Vertical position above the 'Typical M&V CV Requirement' line plot.
    
    (a) = 1-month Metering Periods; (b) = 2-month Metering Periods; (c) = 3-month Metering Periods; (d) = 4-month Metering Periods
    """
    
    fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(8,8),dpi=150)
    titles = ['(a)', '(b)', '(c)', '(d)']
    for i in range(2):
        barplot_data = pd.concat([CVratio_data[['metering period','CV_kWtrain',]].iloc[(i+1)*12-12:(i+1)*12],
                                  CVratio_data[['metering period','CV_kWtest']].iloc[(i+1)*12-12:(i+1)*12]])

        barplot_data['CV'] = barplot_data['CV_kWtrain'].combine_first(barplot_data['CV_kWtest'])
        barplot_data['Dataset'] = ['Train']*12+['Test']*12
    
        sns.barplot(data=barplot_data,x='metering period',y='CV',ax=ax[0][i],hue='Dataset')
    
        ax[0][i].set_ylim(ylim)
        ax[0][i].set_xlabel('Metering Period')
        ax[0][i].set_ylabel('$CV\ [\%]$')
        ax[0][i].set_xticklabels(ax[0][i].get_xticklabels(),fontsize=8,rotation=-30)
        ax[0][i].tick_params(axis='x', which='major', pad=1)
        ax[0][i].text(title_x,title_y,titles[i],weight='bold',ha='center', va='center')
        ax[0][i].text(text_x,15+text_y,"Typical M&V CV Requirement",color='red',fontsize=8)
        
        #Horizontal line for the typical M&V CV limit
        ax[0][i].axhline(y=15, color='red', linestyle='-',lw=2)
    
    for i in range(2,4):
        barplot_data = pd.concat([CVratio_data[['metering period','CV_kWtrain',]].iloc[(i+1)*12-12:(i+1)*12],
                                  CVratio_data[['metering period','CV_kWtest']].iloc[(i+1)*12-12:(i+1)*12]])

        barplot_data['CV'] = barplot_data['CV_kWtrain'].combine_first(barplot_data['CV_kWtest'])
        barplot_data['Dataset'] = ['Train']*12+['Test']*12
    
        sns.barplot(data=barplot_data,x='metering period',y='CV',ax=ax[1][i-2],hue='Dataset')
        ax[1][i-2].set_ylim(ylim)
        ax[1][i-2].set_xlabel('Metering Period')
        ax[1][i-2].set_ylabel('$CV\ [\%]$')
        ax[1][i-2].set_xticklabels(ax[1][i-2].get_xticklabels(),fontsize=8,rotation=-30)
        ax[1][i-2].tick_params(axis='x', which='major', pad=1)
        ax[1][i-2].text(title_x,title_y,titles[i],weight='bold',ha='center', va='center')
        ax[1][i-2].text(text_x,15+text_y,"Typical M&V CV Requirement",color='red',fontsize=8)
    
        #Horizontal line for the typical M&V CV limit
        ax[1][i-2].axhline(y=15, color='red', linestyle='-',lw=2)
    
    fig.suptitle(figtitle)
    plt.tight_layout()
    return fig

**4.11 RMSE barplots function**

In [ ]:
def RMSE_barplots(CVratio_data,ylim,figtitle):
    """This function creates RMSE barplots for 1-month, 2-month, 3-month and 4-month metering samples\n.
    
    Parameters:
    CVratio_data: A ddataframe containing the data to plot.\n
    ylim: y-axes limits entered as a list, [ymin,ymax]\n
    figtitle: Figure title, entered as a string.
    """
    
    fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(8,8),dpi=150)
    titles = ['1 month metering samples', '2 months metering samples', '3 months metering samples', '4 months metering samples']
    for i in range(2):
        barplot_data = pd.concat([CVratio_data[['metering period','RMSE_kWtrain',]].iloc[(i+1)*12-12:(i+1)*12],
                                  CVratio_data[['metering period','RMSE_kWtest']].iloc[(i+1)*12-12:(i+1)*12]])

        barplot_data['RMSE'] = barplot_data['RMSE_kWtrain'].combine_first(barplot_data['RMSE_kWtest'])
        barplot_data['Dataset'] = ['Train']*12+['Test']*12
    
        sns.barplot(data=barplot_data,x='metering period',y='RMSE',ax=ax[0][i],hue='Dataset')
    
        ax[0][i].set_ylim(ylim)
        ax[0][i].set_xlabel('Metering period')
        ax[0][i].set_ylabel('$RMSE\ [kW]$')
        ax[0][i].set_xticklabels(ax[0][i].get_xticklabels(),fontsize=8,rotation=-30)
        ax[0][i].tick_params(axis='x', which='major', pad=1)
        ax[0][i].set_title(titles[i])
    
    for i in range(2,4):
        barplot_data = pd.concat([CVratio_data[['metering period','RMSE_kWtrain',]].iloc[(i+1)*12-12:(i+1)*12],
                                  CVratio_data[['metering period','RMSE_kWtest']].iloc[(i+1)*12-12:(i+1)*12]])

        barplot_data['RMSE'] = barplot_data['RMSE_kWtrain'].combine_first(barplot_data['RMSE_kWtest'])
        barplot_data['Dataset'] = ['Train']*12+['Test']*12
    
        sns.barplot(data=barplot_data,x='metering period',y='RMSE',ax=ax[1][i-2],hue='Dataset')
        ax[1][i-2].set_ylim(ylim)
        ax[1][i-2].set_xlabel('Metering period')
        ax[1][i-2].set_ylabel('$RMSE\ [kW]$')
        ax[1][i-2].set_xticklabels(ax[1][i-2].get_xticklabels(),fontsize=8,rotation=-30)
        ax[1][i-2].tick_params(axis='x', which='major', pad=1)
        ax[1][i-2].set_title(titles[i])
    
    fig.suptitle(figtitle)
    plt.tight_layout()

**4.12 Function for combined plots of CV and RMSE for a selected sample**

In [ ]:
def selected_sample_CV_RMSE_plots(performance_stats_cell,model_title,CV_ylim=[0,50],RMSE_ylim=[0,50],
                                  CV_yticks=[0,10,20,30,40,50],RMSE_yticks=[0,10,20,30,40,50],
                                  ticks_rotation =-30,xticks_font=8,CVtext_x=5,
                                  CVtext_y=1,RMSEtext_x=5,RMSEtext_y=1,legend_font_size=8,title_x=5.5,title_y=-5):
    """
    Parameters\n
        CVtext_x = Horizontal position of the CV_train text on the plot
        CVtext_y = Vertical position above the CV_train horizontal line plot
        RMSEtext_x = Horizontal position of the RMSE_train text on the plot
        RMSEtext_y = Vertical position above the RMSE_train horizontal line plot
        
        (a) = Best 1-month Metering Period; (b) = Best 2-month Metering Period; (c) = Best 3-month Metering Period; 
        (d) = Best 4-month Metering Period 
    """
    
    fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(8,8),dpi=200)

    titles = ['(a)', '(b)','(c)','(d)']
    sample_sets = ['1_month','2_months','3_months','4_months']

    for i in range(len(sample_sets)):
        #Data for the bar plots
        if i < 2:
            sns.barplot(data=performance_stats_cell[sample_sets[i]],x='metering period',y='CV_kWtest',
                        ax=ax[0][i],color="dimgray")
            ax[0][i].set_ylim(CV_ylim)
            ax[0][i].set_xlabel('Test data month')
            ax[0][i].text(title_x,title_y,titles[i],weight='bold',ha='center', va='center')    #Set the position of the title as a bold text
            #Horizontal line for the training data CV
            ax[0][i].axhline(y=performance_stats_cell[sample_sets[i]]['CV_kWtrain'].iloc[0], color ='red',
                             linestyle='-',lw=2)
            ax[0][i].text(CVtext_x,performance_stats_cell[sample_sets[i]]['CV_kWtrain'].iloc[0]+CVtext_y,
                          "$CV_{train}$",color='red',fontsize=8)
            
            #RMSE PLOTS ON SECONDARY AXIS
            ax2 = ax[0][i].twinx()
            sns.lineplot(data=performance_stats_cell[sample_sets[i]],x='metering period',y='RMSE_kWtest',
                         ax=ax2,color="blue",linestyle='-.')
            #Horizontal line for the training data RMSE
            ax2.axhline(y=performance_stats_cell[sample_sets[i]]['RMSE_kWtrain'].iloc[0], color='blue',
                        linestyle='-',lw=2)
            ax2.text(RMSEtext_x,performance_stats_cell[sample_sets[i]]['RMSE_kWtrain'].iloc[0]+RMSEtext_y,
                     "$RMSE_{train}$",color='blue',fontsize=8)
            ax2.set_ylim(RMSE_ylim)
            #Set yticks and tick_params
            if i == 1:
                ax[0][i].set_ylabel('',)
                ax[0][i].tick_params(axis='y', which='both', left=False, right=False, labelleft=False)
                ax2.set_ylabel('$RMSE\ [kW]$')
                ax2.set_yticks(RMSE_yticks)
            else:
                ax[0][i].set_ylabel('$CV\ [\%]$')
                ax[0][i].set_yticks(CV_yticks)
                ax2.set_ylabel('',)
                ax2.tick_params(axis='y', which='both', left=False, right=False,labelright=False)
            
            #Set x labels
            ax[0][i].set_xticklabels(ax[0][i].get_xticklabels(),fontsize=xticks_font,rotation=ticks_rotation)
            ax[0][i].tick_params(axis='x', which='major', pad=1)
            #Legend
            plots = [ax[0][i].get_children()[0], ax2.get_children()[0]]
            labels = ["$CV_{test}$","$RMSE_{test}$"]
            plt.legend(plots, labels, loc="best",fontsize = legend_font_size) 
        else:
            sns.barplot(data=performance_stats_cell[sample_sets[i]],x='metering period',y='CV_kWtest',ax=ax[1][i-2],color="dimgray")
            ax[1][i-2].set_ylim(CV_ylim)
            ax[1][i-2].set_xlabel('Test data month')
            ax[1][i-2].text(title_x,title_y,titles[i],weight='bold',ha='center', va='center') #Manually position the subplot titles
            #Horizontal line for the training data CV
            ax[1][i-2].axhline(y=performance_stats_cell[sample_sets[i]]['CV_kWtrain'].iloc[0], color='red',
                               linestyle='-',lw=2)  
            ax[1][i-2].text(CVtext_x,performance_stats_cell[sample_sets[i]]['CV_kWtrain'].iloc[0]+CVtext_y,
                          "$CV_{train}$",color='red',fontsize=8)
            #RMSE PLOTS ON SECONDARY AXIS
            ax2 = ax[1][i-2].twinx()
            sns.lineplot(data=performance_stats_cell[sample_sets[i]],x='metering period',y='RMSE_kWtest',
                        ax=ax2,color="blue",label="$RMSE_{test}$",linestyle='-.')
            #Horizontal line for the training data RMSE
            ax2.axhline(y=performance_stats_cell[sample_sets[i]]['RMSE_kWtrain'].iloc[0], color='blue',
                        linestyle='-',lw=2)
            ax2.text(RMSEtext_x,performance_stats_cell[sample_sets[i]]['RMSE_kWtrain'].iloc[0]+RMSEtext_y,
                     "$RMSE_{train}$",color='blue',fontsize=8)
            ax2.set_ylim(RMSE_ylim)
            #Set yticks and tick_params
            if i == 3:
                ax[1][i-2].set_ylabel('',)
                ax[1][i-2].tick_params(axis='y', which='both', left=False, right=False, labelleft=False)
                ax2.set_ylabel('$RMSE\ [kW]$')
                ax2.set_yticks(RMSE_yticks)
            else:
                ax[1][i-2].set_ylabel('$CV\ [\%]$')
                ax[1][i-2].set_yticks(CV_yticks)
                ax2.set_ylabel('',)
                ax2.tick_params(axis='y', which='both', left=False, right=False,labelright=False)
            
            #Set x ticks labels
            ax[1][i-2].set_xticklabels(ax[1][i-2].get_xticklabels(),fontsize=xticks_font,rotation=ticks_rotation)
            ax[1][i-2].tick_params(axis='x', which='major', pad=1)
            #Legend
            plots = [ax[1][i-2].get_children()[0],ax2.get_children()[0]]
            labels = ["$CV_{test}$","$RMSE_{test}$"]
            plt.legend(plots, labels, loc="best",fontsize = legend_font_size) 

    fig.suptitle(model_title)
    plt.tight_layout()
    return fig

**4.13 Selected Sample CV Plots function**

In [ ]:
def selected_sample_CVplots(performance_stats_cell,model_title,ylim=[0,50],legend_font_size=8):
    fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(8,8),dpi=150)

    titles = ['Best 1-month metering period', 'Best 2-month metering period',
              'Best 3-month metering period', 'Best 4-month metering period']
    sample_sets = ['1_month','2_months','3_months','4_months']

    for i in range(len(sample_sets)):
        #Data for the bar plots
        if i < 2:
            sns.barplot(data=performance_stats_cell[sample_sets[i]],x='metering period',y='CV_kWtest',ax=ax[0][i],color="dimgray", label="$CV_{test}$")

            ax[0][i].set_ylim(ylim)
            ax[0][i].set_xlabel('Test data month')
            ax[0][i].set_ylabel('$CV\ [\%]$')
            ax[0][i].set_xticklabels(ax[0][i].get_xticklabels(),fontsize=8,rotation=-30)
            ax[0][i].tick_params(axis='x', which='major', pad=1)
            ax[0][i].set_title(titles[i])

            #Horizontal line for the typical M&V CV limit
            ax[0][i].axhline(y=performance_stats_cell[sample_sets[i]]['CV_kWtrain'].iloc[0], color='red',linestyle='-',lw=2,label="$CV_{train}$")
            ax[0][i].legend(fontsize=legend_font_size)

        else:
            sns.barplot(data=performance_stats_cell[sample_sets[i]],x='metering period',y='CV_kWtest',ax=ax[1][i-2],color="dimgray",label="$CV_{test}$")

            ax[1][i-2].set_ylim(ylim)
            ax[1][i-2].set_xlabel('Test data month')
            ax[1][i-2].set_ylabel('$CV\ [\%]$')
            ax[1][i-2].set_xticklabels(ax[1][i-2].get_xticklabels(),fontsize=8,rotation=-30)
            ax[1][i-2].tick_params(axis='x', which='major', pad=1)
            ax[1][i-2].set_title(titles[i])

            #Horizontal line for the typical M&V CV limit
            ax[1][i-2].axhline(y=performance_stats_cell[sample_sets[i]]['CV_kWtrain'].iloc[0], color='red', linestyle='-',lw=2,label="$CV_{train}$")
            ax[1][i-2].legend(fontsize=legend_font_size)

    fig.suptitle(model_title)
    plt.tight_layout()

**4.14 Selected Sample RMSE Plots function**

In [ ]:
def selected_sample_RMSEplots(performance_stats_cell,model_title,ylim=[0,50],legend_font_size=8):
    fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(8,8),dpi=150)

    titles = ['Best 1-month metering sample', 'Best 2-month metering sample',
              'Best 3-month metering sample', 'Best 4-month metering sample']
    sample_sets = ['1_month','2_months','3_months','4_months']

    for i in range(len(sample_sets)):
        #Data for the bar plots
        if i < 2:
            sns.barplot(data=performance_stats_cell[sample_sets[i]],x='metering period',y='RMSE_kWtest',ax=ax[0][i],color="dimgray", label="$RMSE_{kW,test}$")

            ax[0][i].set_ylim(ylim)
            ax[0][i].set_xlabel('Test data month')
            ax[0][i].set_ylabel('$RMSE\ [kW]$')
            ax[0][i].set_xticklabels(ax[0][i].get_xticklabels(),fontsize=8,rotation=-30)
            ax[0][i].tick_params(axis='x', which='major', pad=1)
            ax[0][i].set_title(titles[i])

            #Horizontal line for the typical M&V CV limit
            ax[0][i].axhline(y=performance_stats_cell[sample_sets[i]]['RMSE_kWtrain'].iloc[0], color='red',linestyle='-',lw=2,label="$RMSE_{kW,train}$")
            ax[0][i].legend(fontsize=legend_font_size)

        else:
            sns.barplot(data=performance_stats_cell[sample_sets[i]],x='metering period',y='RMSE_kWtest',ax=ax[1][i-2],color="dimgray",label="$RMSE_{kW,test}$")

            ax[1][i-2].set_ylim(ylim)
            ax[1][i-2].set_xlabel('Test data month')
            ax[1][i-2].set_ylabel('$RMSE\ [kW]$')
            ax[1][i-2].set_xticklabels(ax[1][i-2].get_xticklabels(),fontsize=8,rotation=-30)
            ax[1][i-2].tick_params(axis='x', which='major', pad=1)
            ax[1][i-2].set_title(titles[i])

            #Horizontal line for the typical M&V CV limit
            ax[1][i-2].axhline(y=performance_stats_cell[sample_sets[i]]['RMSE_kWtrain'].iloc[0], color='red', linestyle='-',lw=2,label="$RMSE_{kW,train}$")
            ax[1][i-2].legend(fontsize=legend_font_size)

    fig.suptitle(model_title)
    plt.tight_layout()

**4.15 DOE2 model curves**

In [ ]:
#Model curves
def capft(CHWST,CWRT,a0,a1,a2,a3,a4,a5):
    return a0+a1*CHWST+a2*CHWST**2+a3*CWRT+a4*CWRT**2+a5*CHWST*CWRT

def eirft(CHWST,CWRT,b0,b1,b2,b3,b4,b5):
    return b0+b1*CHWST+b2*CHWST**2+b3*CWRT+b4*CWRT**2+b5*CHWST*CWRT

def eirfplr(PLR,c0,c1,c2):
    return c0+c1*PLR+c2*PLR**2

**4.16 Error variation plot class**

In [ ]:
class error_computation():
    def __init__(self,chiller_data):
        """
        chiller_data: The dataframe containing the chiller measured data
        """
        self.chiller_data = chiller_data

    def temp_indep_model(self,model):
        model_vars = self.chiller_data[['Load','ELE']]
        model_vars['y'] = FeatureGenerator.TempIndep(model_vars).y_label()
        X = FeatureGenerator.TempIndep(model_vars).X_features()
        model_vars['y_pred'] = model.predict(X)
        model_vars['ELE_pred'] = ModelPower.temp_indep_model(model_vars['Load'],model_vars['y_pred'])
        model_vars['abs_error'] = abs(model_vars['ELE']-model_vars['ELE_pred'])
        model_vars['Time_range'] = self.chiller_data['Time_range']
        return model_vars
    
    def quasi_empirical_model(self,model):
        model_vars = self.chiller_data[['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']
        model_vars['y'] = FeatureGenerator.QuasiEmpirical(model_vars).y_label()
        X = FeatureGenerator.QuasiEmpirical(model_vars).X_features()
        model_vars['y_pred'] = model.predict(X)
        model_vars['ELE_pred'] = ModelPower(model_vars).quasi_empirical_model(model_vars['Load'],model_vars['y_pred'],
                                                                             model_vars['CWRT'],model_vars['CHWST'])
        model_vars['abs_error'] = abs(model_vars['ELE']-model_vars['ELE_pred'])
        model_vars['Time_range'] = self.chiller_data['Time_range']
        return model_vars
    
    def GN_fundamental_model(self,model):
        model_vars = self.chiller_data[['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']
        model_vars['y'] = FeatureGenerator.GNFundamental(model_vars).y_label()
        X = FeatureGenerator.GNFundamental(model_vars).X_features()
        model_vars['y_pred'] = model.predict(X)
        model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['y_pred'],
                                                                             model_vars['CWRT'],model_vars['CHWST'])
        model_vars['abs_error'] = abs(model_vars['ELE']-model_vars['ELE_pred'])
        model_vars['Time_range'] = self.chiller_data['Time_range']
        return model_vars
    
    def GN_fundamental_DS_model(self,model):
        model_vars = self.chiller_data[['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']
        model_vars['y'] = FeatureGenerator.GNFundamental_DS(model_vars).y_label()
        X = FeatureGenerator.GNFundamental_DS(model_vars).X_features()
        model_vars['y_pred'] = model.predict(X)
        model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['y_pred'],
                                                                             model_vars['CWRT'],model_vars['CHWST'])
        model_vars['abs_error'] = abs(model_vars['ELE']-model_vars['ELE_pred'])
        model_vars['Time_range'] = self.chiller_data['Time_range']
        return model_vars
    
    def GN_split_Qleak_model(self,model):
        model_vars = self.chiller_data[['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']
        model_vars['y'] = FeatureGenerator.GNSplit_Qleak(model_vars).y_label()
        X = FeatureGenerator.GNSplit_Qleak(model_vars).X_features()
        model_vars['y_pred'] = model.predict(X)
        model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['y_pred'],
                                                                             model_vars['CWRT'],model_vars['CHWST'])
        model_vars['abs_error'] = abs(model_vars['ELE']-model_vars['ELE_pred'])
        model_vars['Time_range'] = self.chiller_data['Time_range']
        return model_vars
    
    def GN_Foliaco_model(self,model):
        model_vars = self.chiller_data[['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']
        model_vars['y'] = FeatureGenerator.GNFoliaco(model_vars).y_label()
        X = FeatureGenerator.GNFoliaco(model_vars).X_features()
        model_vars['y_pred'] = model.predict(X)
        model_vars['ELE_pred'] = ModelPower(model_vars).GN_Foliaco_model(model_vars['Load'],model_vars['y_pred'],
                                                                             model_vars['CWRT'],model_vars['CHWST'])
        model_vars['abs_error'] = abs(model_vars['ELE']-model_vars['ELE_pred'])
        model_vars['Time_range'] = self.chiller_data['Time_range']
        return model_vars
        
    def biquadratic_model(self,model):
        model_vars = self.chiller_data[['CWET','Load','ELE']]
        model_vars.columns = ['CWRT','Load','ELE']
        model_vars['y'] = FeatureGenerator.BiQuadratic(model_vars).y_label()
        X = FeatureGenerator.BiQuadratic(model_vars).X_features()
        model_vars['y_pred'] = model.predict(X)
        model_vars['ELE_pred'] = ModelPower(model_vars).biquadratic_model(model_vars['Load'],model_vars['y_pred'])
        model_vars['abs_error'] = abs(model_vars['ELE']-model_vars['ELE_pred'])
        model_vars['Time_range'] = self.chiller_data['Time_range']
        return model_vars
    
    def multivariate_model(self,model):
        model_vars = self.chiller_data[['CWET','CHLR_RCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWRT','Load','ELE']
        model_vars['y'] = FeatureGenerator.MultiVariate(model_vars).y_label()
        X = FeatureGenerator.MultiVariate(model_vars).X_features()
        model_vars['y_pred'] = model.predict(X)
        model_vars['ELE_pred'] = ModelPower(model_vars).multivariate_model(model_vars['Load'],model_vars['y_pred'])
        model_vars['abs_error'] = abs(model_vars['ELE']-model_vars['ELE_pred'])
        model_vars['Time_range'] = self.chiller_data['Time_range']
        return model_vars
    
    def CP_model(self,model):
        model_vars = self.chiller_data[['ELE','Twb']]
        model_vars['ELE_pred'] = four_CP(model_vars['Twb'].to_numpy(),model[0],model[1],model[2],model[3])
        #The function elec is defined in in the 4CP model section
        model_vars['abs_error'] = abs(model_vars['ELE']-model_vars['ELE_pred'])
        model_vars['Time_range'] = self.chiller_data['Time_range']
        return model_vars
    
    def refCurves_model(self,model):
        model_vars = self.chiller_data[['Time_range','CHLR_LCHWT','CWET','Load','ELE']]
        model_vars.columns = ['Time_range','CHWST','CWRT','Load','ELE']

        #Units conversion to kW and C
        model_vars['CHWST'] = (model_vars['CHWST']-32)*5/9
        model_vars['CWRT'] = (model_vars['CWRT']-32)*5/9
        #DOE2 model variables
        model_vars['CAPFT'] = capft(model_vars['CHWST'],model_vars['CWRT'],model['CAPFT'][0],model['CAPFT'][1],
                                   model['CAPFT'][2],model['CAPFT'][3],model['CAPFT'][4],model['CAPFT'][5])
        model_vars['EIRFT'] = eirft(model_vars['CHWST'],model_vars['CWRT'],model['EIRFT'][0],model['EIRFT'][1],
                                    model['EIRFT'][2],model['EIRFT'][3],model['EIRFT'][4],model['EIRFT'][5])
        model_vars['PLR'] = model_vars['Load']/(model['Qref']*model_vars['CAPFT'])
        model_vars['EIRFPLR'] = eirfplr(model_vars['PLR'],model['EIRFPLR'][0],model['EIRFPLR'][1],model['EIRFPLR'][2])
        
        model_vars['ELE_pred'] = model_vars['CAPFT']*model_vars['EIRFT']*model_vars['EIRFPLR']*model['Pref']
        model_vars['abs_error'] = abs(model_vars['ELE']-model_vars['ELE_pred'])
        return model_vars

**4.17 Error variation plot: Best sample per metering period**

In [ ]:
def error_variation_plot(error_data,model_title,power_ylim=[0,400],power_ticks=np.arange(0,400+1,80),
                         twb_ylim=[0,100],twb_ticks=np.arange(0,100+1,20),figsize=(10,14),dpi=100,legend_font_size=8):
    titles = ['Best 1-month metering sample', 'Best 2-month metering sample',
              'Best 3-month metering sample', 'Best 4-month metering sample']
    sample_sets = list(error_data.keys())
    
    fig,ax = plt.subplots(ncols=1,nrows=4,figsize=figsize,dpi=dpi)

    for i in range(len(error_data)):
        sns.lineplot(data=error_data[sample_sets[i]],x='Time_range',y='ELE',ax=ax[i],color='green')
        ax[i].set_ylim(power_ylim)
        ax[i].set_xlabel('Time')
        ax[i].set_ylabel('Measured power or Absolute error [kW]')
        ax[i].set_xlim(datetime(2019, 1, 1, 0, 0, 0), datetime(2020, 1, 1, 0, 0, 0))
        ax[i].set_title(titles[i])
        ax[i].set_yticks(power_ticks)
        sns.lineplot(data=error_data[sample_sets[i]],x='Time_range',y='abs_error',ax=ax[i],color='red')
        
        ax2 = ax[i].twinx()
        sns.lineplot(data=weather,x='Time_range',y='Twb',ax=ax2,color='skyblue')
        ax2.set_ylim(twb_ylim)
        ax2.set_ylabel('$T_{wb}\ [^{o}F]$')
        ax2.set_yticks(twb_ticks)
        
        #Legend
        plots = [ax[i].get_children()[0],ax[i].get_children()[1], ax2.get_children()[0]]
        labels = ["Measured power","Absolute error","$T_{wb}$"]
        plt.legend(plots, labels, loc="best",fontsize=legend_font_size)

    fig.suptitle(model_title)
    plt.tight_layout()

**4.18 Error variation plot: Overall best sample and metering period**

In [ ]:
def error_overall_best(error_data,model_title,power_ylim=[0,400],power_ticks=np.arange(0,400+1,80),
                       twb_ylim=[0,100],twb_ticks=np.arange(0,100+1,20),figsize=(6,10),dpi=150,legend_font_size=8):
    
    plt.figure(figsize=figsize,dpi=dpi)
    
    ax = sns.lineplot(data=error_data,x='Time_range',y='ELE',color='green')
    ax.set_ylim(power_ylim)
    ax.set_xlabel('Time')
    ax.set_ylabel('Measured power or Absolute error [kW]')
    ax.set_xlim(datetime(2019, 1, 1, 0, 0, 0), datetime(2020, 1, 1, 0, 0, 0))
    ax.set_yticks(power_ticks)
    ax = sns.lineplot(data=error_data,x='Time_range',y='abs_error',color='red')
    
    ax2 = ax.twinx()
    sns.lineplot(data=weather,x='Time_range',y='Twb',ax=ax2,color='skyblue')
    ax2.set_ylim(twb_ylim)
    ax2.set_ylabel('$T_{wb}\ [^{o}F]$')
    ax2.set_yticks(twb_ticks)
    
     #Legend
    plots = [ax.get_children()[0],ax.get_children()[1],ax2.get_children()[0]]
    labels = ["Measured power","Absolute error","$T_{wb}$"]
    plt.legend(plots, labels, loc="best",fontsize=legend_font_size)
    ax.set_title(model_title)

**4.19 dfs to contain the CV_train and CV_test for comparison of all models**

In [ ]:
# First level headers
headers_1 = ['temp_ind','quasi_emp','GN_no_interc','GN_w_interc','GN_DS','GN_split','biquadratic','multivariate','4CP','RefCurves']

# Second level headers
headers_2 = ['CV_train', 'CV_test']

# Create a MultiIndex object from the product of the two arrays
multi_headers = pd.MultiIndex.from_product([headers_1, headers_2])
comparison_df = pd.DataFrame(columns=multi_headers,index=['sample{}'.format(str(i+1)) for i in range(12)])

comparison_dfs = {'1_month':comparison_df.copy(),'2_months':comparison_df.copy(),'3_months':comparison_df.copy(),
                  '4_months':comparison_df.copy()}

comparison_dfs['3_months']

**4.20 dfs to summarize the model performances:**  

**Stats to include:**
1. Min CV_test*CV_train value
2. Best sampling period with min CV_net
3. Average CV_train*CV_test: Include {Average,median,max CV_train*CV_test}

In [ ]:
# First level headers
headers_1 = ['1 month', '2 months', '3 months', '4 months']

models = ['temp_ind','quasi_emp','GN_no_interc','GN_w_interc','GN_DS','GN_split','GN_Foliaco','biquadratic','multivariate',
           '4CP','RefCurves']

# Second level headers
headers_2 = ['Best metering period', 'CV_kW_net']

# Create a MultiIndex object from the product of the two arrays
multi_headers = pd.MultiIndex.from_product([headers_1, headers_2])
best_periods = pd.DataFrame(columns=multi_headers,index= models)

best_periods

**Overall Performance**

In [ ]:
# First level headers
headers_1 = ['1 month', '2 months', '3 months', '4 months']

models = ['temp_ind','quasi_emp','GN_no_interc','GN_w_interc','GN_DS','GN_split','GN_Foliaco','biquadratic','multivariate',
           '4CP','RefCurves']

# Second level headers
headers_2 = ['Min','Max','Average', 'Median']

# Create a MultiIndex object from the product of the two arrays
multi_headers = pd.MultiIndex.from_product([headers_1, headers_2])
performance_averages = pd.DataFrame(columns=multi_headers,index= models)

performance_averages

**Psi function = For determining extent of model extrapolation** 

In [ ]:
def psi(CVratio_data,Twb_max,Twb_min):
    CVratio_data['psi'] = np.zeros(48)
    
    for i in range(48):
        if CVratio_data['Sample_set'][i] == '1_month':
            Twb_sample_max = metering_sample_sets['1_month']['sample{n}'.format(n=str(CVratio_data.index[i]+1))]['Twb'].max()
            Twb_sample_min = metering_sample_sets['1_month']['sample{n}'.format(n=str(CVratio_data.index[i]+1))]['Twb'].min()
            CVratio_data['psi'][i]  = (Twb_max-Twb_sample_max)+(Twb_sample_min-Twb_min)
        
        elif CVratio_data['Sample_set'][i] == '2_months':
            Twb_sample_max = metering_sample_sets['2_months']['sample{n}'.format(n=str(CVratio_data.index[i-12]+1))]['Twb'].max()
            Twb_sample_min = metering_sample_sets['2_months']['sample{n}'.format(n=str(CVratio_data.index[i-12]+1))]['Twb'].min()
            CVratio_data['psi'][i]  = (Twb_max-Twb_sample_max)+(Twb_sample_min-Twb_min)
    
        elif CVratio_data['Sample_set'][i] == '3_months':
            Twb_sample_max = metering_sample_sets['3_months']['sample{n}'.format(n=str(CVratio_data.index[i-24]+1))]['Twb'].max()
            Twb_sample_min = metering_sample_sets['3_months']['sample{n}'.format(n=str(CVratio_data.index[i-24]+1))]['Twb'].min()
            CVratio_data['psi'][i]  = (Twb_max-Twb_sample_max)+(Twb_sample_min-Twb_min)
        
        else:
            Twb_sample_max = metering_sample_sets['4_months']['sample{n}'.format(n=str(CVratio_data.index[i-36]+1))]['Twb'].max()
            Twb_sample_min = metering_sample_sets['4_months']['sample{n}'.format(n=str(CVratio_data.index[i-36]+1))]['Twb'].min()
            CVratio_data['psi'][i]  = (Twb_max-Twb_sample_max)+(Twb_sample_min-Twb_min)   

    return CVratio_data['psi']

# **5.2 Temperature-independent model**

**MODEL EQUATION**

$\frac{1}{COP} = a_{1}.\frac{1}{Q_{ev}} + a_{0}$

**5.2.1 Computations**

In [ ]:
combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['Load','ELE']]

        #Model training features 
        X_train = FeatureGenerator.TempIndep(model_vars).X_features()
        y_train = FeatureGenerator.TempIndep(model_vars).y_label()
        
        #Model regression
        model = Regression(X_train,y_train).with_intercept()

        #Model regression train stats
        model_vars['train_pred'] =  ypred_and_stats.y_pred(model,X_train)
        train_ystats,model_vars = train_regression_stats(model,model_vars,X_train,y_train)
        stats = np.append(stats,train_ystats)
        #Predicted ELE: Train data
        model_vars['ELE_pred'] = ModelPower.temp_indep_model(model_vars['Load'],model_vars['train_pred'])
        #Model power train stats
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)

        #TEST DATA MODEL PREDICTION
        test_vars = test_sample_sets[sample_set][sample][['Load','ELE']]
        
        #Test data features
        X_test = FeatureGenerator.TempIndep(test_vars).X_features()
        y_test = FeatureGenerator.TempIndep(test_vars).y_label()
        
        #Model regression stats: Test data
        test_vars['test_pred'] = ypred_and_stats.y_pred(model,X_test)
        test_ystats,test_vars = test_regression_stats(model,test_vars,X_test,y_test)
        stats = np.append(stats,test_ystats)

        #Power prediction stats: Test data
        test_vars['ELE_pred'] = ModelPower.temp_indep_model(test_vars['Load'],test_vars['test_pred'])
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
        #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('temp_ind','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('temp_ind','CV_test')] = test_kWstats[2]
    
    #Combined stats df
    combined_stats_sets[sample_set] = stats_combiner(combined_stats_sets[sample_set])
    
# CVratio data for plotting
CVratio_data_temp_independent = CVratio_generator(combined_stats_sets)

CVratio_data_temp_independent['psi'] = psi(CVratio_data_temp_independent,Twb_max,Twb_min)

CVratio_data_temp_independent

**5.2.2 CV barplots: Temperature-independent model**

In [ ]:
CV_barplots(CVratio_data_temp_independent,[0,60],figtitle='Temperature-independent model',title_x=5.5,title_y=-15)

**5.2.3 RMSE barplots: Temperature-independent model**

In [ ]:
RMSE_barplots(CVratio_data_temp_independent,[0,100],'Temperature-independent model')

**5.2.4 Monthly variation of test data CV and RMSE: Best Sample**

In [ ]:
best_samples = {'1_month':'sample7','2_months':'sample7','3_months':'sample4','4_months':'sample4'}
stats_temp_indep = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_temp_indep = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['Load','ELE']]
    #Extract only the features to be used for regression
    #Model training features
    X_train = FeatureGenerator.TempIndep(model_vars).X_features()
    y_train = FeatureGenerator.TempIndep(model_vars).y_label()

    #Model regression
    model = Regression(X_train, y_train).with_intercept()
    best_models_temp_indep[sample_set] = model

    #y-predictions and model stats:Training data
    model_vars['train_pred'] = model.predict(X_train)

    MAE_ytrain = mean_absolute_error(y_train,model_vars['train_pred'])
    MAE_perc = mean_absolute_percentage_error(y_train,model_vars['train_pred'])
    MBE_ytrain = MAE_ytrain*100/np.mean(y_train)
    MSE_ytrain = mean_squared_error(y_train,model_vars['train_pred'])
    RMSE_ytrain = np.sqrt(MSE_ytrain)
    CV_ytrain = RMSE_ytrain*100/np.mean(y_train)

    #POWER PREDICTION STATS: TRAINING DATA
    model_vars['ELE_pred'] = ModelPower.temp_indep_model(model_vars['Load'],model_vars['train_pred'])
    
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])


    #y-predictions and model stats: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['Load','ELE']]

    #Extract only the features to be used for regression
    X_test = FeatureGenerator.TempIndep(test_vars).X_features()
    y_test = FeatureGenerator.TempIndep(test_vars).y_label()
    
    test_vars['y'] = y_test
    test_vars['Time_range'] = test_sample_sets[sample_set][best_sample]['Time_range']
    test_vars['month'] = test_sample_sets[sample_set][best_sample]['month']
    test_vars['test_pred'] = model.predict(X_test)

    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        # Model STATS for every test data month
        MAE_ytest = mean_absolute_error(test_vars_month['y'],test_vars_month['test_pred'])
        MBE_ytest = MAE_ytest*100/np.mean(test_vars_month['y'])
        MSE_ytest = mean_squared_error(test_vars_month['y'],test_vars_month['test_pred'])
        RMSE_ytest = np.sqrt(MSE_ytest)
        CV_ytest = RMSE_ytest*100/np.mean(test_vars_month['y'])

        #Power prediction stats: Monthly Test data
        test_vars_month['ELE_pred'] = ModelPower.temp_indep_model(test_vars_month['Load'],test_vars_month['test_pred'])
        
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])
    
    stats_temp_indep[sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    #Creating a new column containing the index values
    stats_temp_indep[sample_set] = stats_temp_indep[sample_set].reset_index().rename(columns={'index': 'metering period'})

stats_temp_indep['4_months']

In [ ]:
selected_sample_CVplots(stats_temp_indep,"Temperature-independent model",[0,65])

In [ ]:
selected_sample_RMSEplots(stats_temp_indep,"Temperature-independent model",[0,50])

In [ ]:
selected_sample_CV_RMSE_plots(stats_temp_indep,model_title="Temperature-independent model",CV_ylim=[0,70],
                              CV_yticks=np.arange(0,70+1,10),RMSE_ylim=[0,70],RMSE_yticks=np.arange(0,70+1,10),
                              ticks_rotation=-30,CVtext_y=1,RMSEtext_y=1,title_x=5.5,title_y=-15)

**5.2.5 Variation prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chiller_data=chillerdata).temp_indep_model(best_models_temp_indep[sample_set])

# PLOTS
error_variation_plot(error_data,"Temperature-independent model")

**5.2.6 Error variation: Best overall sample** 

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).temp_indep_model(best_models_temp_indep[best_sample_set])

# PLOTS
error_overall_best(error_data,"Temperature-independent model",figsize=(10,6))

# **5.3 Quasi-empirical GN model (1994/95)**

**MODEL EQUATION**

$(\frac{1}{COP}+1).Q_{ev}-Q_{ev}.\frac{T_{cwi}}{T_{cho}} = a_{0}+a_{1}.T_{cwi}+a_{2}.\frac{T_{cwi}}{T_{cho}}$

$y = a_{o}+a_{1}.x_{1}+a_{2}.x_{2}$

**5.3.1 Computations**

In [ ]:
combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']

        #Model training features
        y_train = FeatureGenerator.QuasiEmpirical(model_vars).y_label()
        X_train = FeatureGenerator.QuasiEmpirical(model_vars).X_features()

        #Model regression
        model = Regression(X_train,y_train).with_intercept()
        
        #Model regression train stats
        model_vars['train_pred'] =  model.predict(X_train)
        train_ystats,model_vars = train_regression_stats(model,model_vars,X_train,y_train)
        stats = np.append(stats,train_ystats)

        #Predicted ELE: Train data
        model_vars['ELE_pred'] = ModelPower(model_vars).quasi_empirical_model(model_vars['Load'],model_vars['train_pred'],
                                                                              model_vars['CWRT'],model_vars['CHWST'])
        #Model power train stats
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)

        #TEST DATA MODEL PREDICTION
        test_vars = test_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        test_vars.columns =  ['CWRT','CHWST','Load','ELE']
        
        #Test data features
        y_test = FeatureGenerator.QuasiEmpirical(test_vars).y_label()
        X_test = FeatureGenerator.QuasiEmpirical(test_vars).X_features()
        
        #Model regression stats: Test data
        test_vars['test_pred'] =  model.predict(X_test)
        test_ystats,model_vars = test_regression_stats(model,test_vars,X_test,y_test)
        stats = np.append(stats,test_ystats)

        #Power prediction stats: Test data
        test_vars['ELE_pred'] = ModelPower(test_vars).quasi_empirical_model(test_vars['Load'],test_vars['test_pred'],
                                                                              test_vars['CWRT'],test_vars['CHWST'])
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
        #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('quasi_emp','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('quasi_emp','CV_test')] = test_kWstats[2]

    #Combined stats df
    combined_stats_sets[sample_set] = stats_combiner(combined_stats_sets[sample_set])
    
# CVratio data for plotting
CVratio_data_quasi_empirical = CVratio_generator(combined_stats_sets)

CVratio_data_quasi_empirical['psi'] = psi(CVratio_data_quasi_empirical,Twb_max,Twb_min)
CVratio_data_quasi_empirical

**5.3.2 CV barplots: Quasi-empirical model**

In [ ]:
CV_barplots(CVratio_data_quasi_empirical,[0,50],"Quasi-empirical GN model")

**5.3.3 RMSE barplots: Quasi-empirical model**

In [ ]:
RMSE_barplots(CVratio_data_quasi_empirical,[0,90],"Quasi-empirical GN model")

**5.3.4 Monthly variation of test data CV and RMSE: Best Sample**

In [ ]:
best_samples = {'1_month':'sample10','2_months':'sample9','3_months':'sample9','4_months':'sample8'}
#Best samples selected as those with minimum CV_net

stats_quasi_empirical = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_quasi_empirical = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    model_vars.columns = ['CWRT','CHWST','Load','ELE']
    #Extract only the features to be used for regression
    y_train = FeatureGenerator.QuasiEmpirical(model_vars).y_label()
    X_train = FeatureGenerator.QuasiEmpirical(model_vars).X_features()

    #Model regression
    model = Regression(X_train, y_train).with_intercept()
    best_models_quasi_empirical[sample_set] = model
    
    #y-predictions and model stats:Training data
    model_vars['train_pred'] = model.predict(X_train)

    MAE_ytrain = mean_absolute_error(y_train,model_vars['train_pred'])
    MAE_perc = mean_absolute_percentage_error(y_train,model_vars['train_pred'])
    MBE_ytrain = MAE_ytrain*100/np.mean(y_train)
    MSE_ytrain = mean_squared_error(y_train,model_vars['train_pred'])
    RMSE_ytrain = np.sqrt(MSE_ytrain)
    CV_ytrain = RMSE_ytrain*100/np.mean(y_train)

    #POWER PREDICTION STATS: TRAINING DATA
    model_vars['ELE_pred'] = ModelPower(model_vars).quasi_empirical_model(model_vars['Load'],model_vars['train_pred'],
                                                                        model_vars['CWRT'],model_vars['CHWST'])
    
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])


    #y-predictions and model stats: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    test_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_test = FeatureGenerator.QuasiEmpirical(test_vars).y_label()
    X_test = FeatureGenerator.QuasiEmpirical(test_vars).X_features()
    
    test_vars['y'] = y_test
    test_vars['Time_range'] = test_sample_sets[sample_set][best_sample]['Time_range']
    test_vars['month'] = test_sample_sets[sample_set][best_sample]['month']
    test_vars['test_pred'] = model.predict(X_test)

    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        # Model STATS for every test data month
        MAE_ytest = mean_absolute_error(test_vars_month['y'],test_vars_month['test_pred'])
        MBE_ytest = MAE_ytest*100/np.mean(test_vars_month['y'])
        MSE_ytest = mean_squared_error(test_vars_month['y'],test_vars_month['test_pred'])
        RMSE_ytest = np.sqrt(MSE_ytest)
        CV_ytest = RMSE_ytest*100/np.mean(test_vars_month['y'])

        #Power prediction stats: Monthly Test data
        test_vars_month['ELE_pred'] = ModelPower(test_vars_month).quasi_empirical_model(test_vars_month['Load'],test_vars_month['test_pred'],
                                                                        test_vars_month['CWRT'],test_vars_month['CHWST'])
        
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])
    
    stats_quasi_empirical[sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
     #Creating a new column containing the index values
    stats_quasi_empirical[sample_set] = stats_quasi_empirical[sample_set].reset_index().rename(columns={'index':'metering period'})

stats_quasi_empirical['4_months']

In [ ]:
selected_sample_CVplots(stats_quasi_empirical,"Quasi-empirical model",[0,40])

In [ ]:
selected_sample_RMSEplots(stats_quasi_empirical,"Quasi-empirical model",[0,30])

In [ ]:
selected_sample_CV_RMSE_plots(stats_quasi_empirical,"Quasi-empirical model",CV_ylim=[0,30],
                              CV_yticks=np.arange(0,30+1,10),RMSE_ylim=[0,30],RMSE_yticks=np.arange(0,30+1,10),
                              ticks_rotation=-30)

**5.3.6 Variation prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chillerdata).quasi_empirical_model(best_models_quasi_empirical[sample_set])

# PLOTS
error_variation_plot(error_data,"Quasi-empirical GN model")

**5.3.7 Error variation: Overall best sample**

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).quasi_empirical_model(best_models_quasi_empirical[best_sample_set])

# PLOTS
error_overall_best(error_data,"Quasi-empirical GN model",figsize=(10,6))

# **5.4 GN model with zero intercept (2000)**

**MODEL EQUATION**

$(\frac{1}{COP}+1).\frac{T_{cho}}{T_{cwi}}-1 = a_{1}.\frac{T_{cho}}{Q_{ev}}+a_{2}.\frac{T_{cwi}-T_{cho}}{T_{cwi}.Q_{ev}}+
a_{3}.\frac{Q_{ev}}{T_{cwi}}(1+\frac{1}{COP})$

$y = a_{1}.x_{1}+a_{2}.x_{2}+a_{3}x_{3}$

**5.4.1 Computations**

In [ ]:
combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']

        #Model training features
        y_train = FeatureGenerator.GNFundamental(model_vars).y_label()
        X_train = FeatureGenerator.GNFundamental(model_vars).X_features()

        #Model regression
        model = Regression(X_train,y_train).no_intercept()

        #Model regression train stats
        model_vars['train_pred'] = model.predict(X_train)
        train_ystats,model_vars = train_regression_stats(model,model_vars,X_train,y_train)
        stats = np.append(stats,train_ystats)
        #Predicted ELE: Train data
        model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['train_pred'],
                                                                              model_vars['CWRT'],model_vars['CHWST'])

        #Model power train stats
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)

        #TEST DATA MODEL PREDICTION
        test_vars = test_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        test_vars.columns = ['CWRT','CHWST','Load','ELE']
        
        #Test data features
        y_test = FeatureGenerator.GNFundamental(test_vars).y_label()
        X_test = FeatureGenerator.GNFundamental(test_vars).X_features()
        
        #Model regression stats: Test data
        test_vars['test_pred'] = model.predict(X_test)
        test_ystats,test_vars = test_regression_stats(model,test_vars,X_test,y_test)
        stats = np.append(stats,test_ystats)

        #Power prediction stats: Test data
        test_vars['ELE_pred'] = ModelPower(test_vars).GN_fundamental_model(test_vars['Load'],test_vars['test_pred'],
                                                                              test_vars['CWRT'],test_vars['CHWST'])
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
        #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('GN_no_interc','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('GN_no_interc','CV_test')] = test_kWstats[2]

    #Combined stats df
    combined_stats_sets[sample_set] = stats_combiner(combined_stats_sets[sample_set])

# CVratio data for plotting
CVratio_data_GN_no_inter = CVratio_generator(combined_stats_sets)                                                                   


CVratio_data_GN_no_inter['psi'] = psi(CVratio_data_GN_no_inter,Twb_max,Twb_min)
CVratio_data_GN_no_inter 

**5.4.2 CV barplots: Fundamental GN model with zero intercept**

In [ ]:
CV_fig = CV_barplots(CVratio_data_GN_no_inter,[0,30],figtitle="", text_y=0.5,
                    title_x=5.5,title_y=-7)

CV_fig.savefig('CH1_Performance_deterioration_GN_no_interc.png',bbox_inches='tight')

**5.4.3 RMSE barplots: Fundamental GN model with zero intercept**

In [ ]:
RMSE_barplots(CVratio_data_GN_no_inter,[0,60],figtitle="Fundamental GN model with zero intercept")

**5.4.4 Monthly variation of test data CV and RMSE: Best Sample**

In [ ]:
best_samples = {'1_month':'sample10','2_months':'sample5','3_months':'sample4','4_months':'sample4'}
stats_GN_no_interc = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_GN_no_interc = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    model_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_train = FeatureGenerator.GNFundamental(model_vars).y_label()
    X_train = FeatureGenerator.GNFundamental(model_vars).X_features()

    #Model regression
    model = Regression(X_train, y_train).no_intercept()
    best_models_GN_no_interc[sample_set] = model

    #y-predictions and model stats:Training data
    model_vars['train_pred'] = model.predict(X_train)

    MAE_ytrain = mean_absolute_error(y_train,model_vars['train_pred'])
    MAE_perc = mean_absolute_percentage_error(y_train,model_vars['train_pred'])
    MBE_ytrain = MAE_ytrain*100/np.mean(y_train)
    MSE_ytrain = mean_squared_error(y_train,model_vars['train_pred'])
    RMSE_ytrain = np.sqrt(MSE_ytrain)
    CV_ytrain = RMSE_ytrain*100/np.mean(y_train)

    #POWER PREDICTION STATS: TRAINING DATA
    model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['train_pred'],
                                                                              model_vars['CWRT'],model_vars['CHWST'])
    
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])


    #y-predictions and model stats: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    test_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_test = FeatureGenerator.GNFundamental(test_vars).y_label()
    X_test = FeatureGenerator.GNFundamental(test_vars).X_features()
    
    test_vars['y'] = y_test
    test_vars['Time_range'] = test_sample_sets[sample_set][best_sample]['Time_range']
    test_vars['month'] = test_sample_sets[sample_set][best_sample]['month']
    test_vars['test_pred'] = model.predict(X_test)

    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        # Model STATS for every test data month
        MAE_ytest = mean_absolute_error(test_vars_month['y'],test_vars_month['test_pred'])
        MBE_ytest = MAE_ytest*100/np.mean(test_vars_month['y'])
        MSE_ytest = mean_squared_error(test_vars_month['y'],test_vars_month['test_pred'])
        RMSE_ytest = np.sqrt(MSE_ytest)
        CV_ytest = RMSE_ytest*100/np.mean(test_vars_month['y'])

        #Power prediction stats: Monthly Test data
        test_vars_month['ELE_pred'] = ModelPower(test_vars_month).GN_fundamental_model(test_vars_month['Load'],test_vars_month['test_pred'],
                                                                        test_vars_month['CWRT'],test_vars_month['CHWST'])
        
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])
    
    stats_GN_no_interc[sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    #Creating a new column containing the index values
    stats_GN_no_interc[sample_set] = stats_GN_no_interc[sample_set].reset_index().rename(columns={'index': 'metering period'})

stats_GN_no_interc['4_months']

In [ ]:
selected_sample_CVplots(stats_GN_no_interc,'Fundamental GN model without intercept',[0,30])

In [ ]:
selected_sample_RMSEplots(stats_GN_no_interc,'Fundamental GN model without intercept',[0,20])

In [ ]:
fig_CV_RMSE = selected_sample_CV_RMSE_plots(stats_GN_no_interc,"Fundamental GN model without intercept",CV_ylim=[0,20],
                              CV_yticks=np.arange(0,20+1,4),RMSE_ylim=[0,20],RMSE_yticks=np.arange(0,20+1,4),
                              ticks_rotation=-30,CVtext_y=0.5,RMSEtext_y=0.5)
fig_CV_RMSE.savefig('CH1_Best_samples_GN_no_interc.png',bbox_inches='tight')

**5.4.5 Variation of prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chillerdata).GN_fundamental_model(best_models_GN_no_interc[sample_set])

# PLOTS
error_variation_plot(error_data,"Fundamental GN model without intercept")

**5.4.6 Error variation: Overall best sample**

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).GN_fundamental_model(best_models_GN_no_interc[best_sample_set])

# PLOTS
error_overall_best(error_data,"Fundamental GN model without intercept",figsize=(10,6),power_ticks=np.arange(0,400+1,40),
                  twb_ticks=np.arange(0,100+1,10))

# 5.5 GN model DS = f(Tcho,PLR)

* **W. Jiang and T.A. Reddy (2003):**
    * The GN model was not originally developed for chillers with VSD or inlet vanes capacity control
    * ${\Delta}S$ is expected to varry with chiller PLR
    * The authors investigated the following expression:
    * ${\Delta}S = a_{1}+a_{2}\bigg(\frac{Q_{ev}}{Q_{ev,rated}}\bigg)$
    
    * I **preliminarily investigated** the effect of including Tchi with a lab chiller data from the RP-1043 ASHRAE research project
    * The following expression showed the best performance:
        * ${\Delta}S = a_{1}+a_{2}\bigg(\frac{T_{cho}}{T_{cho,des}}\bigg)+a_{3}\bigg(\frac{Q_{ev}}{Q_{ev,rated}}\bigg)$

**REFORMULATED EQUATION**

$\bigg(\frac{1}{COP}+1\bigg).\frac{T_{cho}}{T_{cwi}}-1 = \bigg(a_{1}+a_{2}.\frac{T_{cho}}{T_{cho,des}}+a_{3}\frac{Q_{ev}}{Q_{ev,rated}}\bigg).\frac{T_{cho}}{Q_{ev}}+a_{4}.\frac{T_{cwi}-T_{cho}}{T_{cwi}.Q_{ev}}+a_{5}.\frac{Q_{ev}}{T_{cwi}}\bigg(1+\frac{1}{COP}\bigg)$

$y = a_{1}.x_{1}+a_{2}.x_{2}+a_{3}x_{3}+a_{4}x_{4}+a_{5}x_{5}$

**5.5.1 Computations**

In [ ]:
combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']

        #Model training features
        y_train = FeatureGenerator.GNFundamental_DS(model_vars).y_label()
        X_train = FeatureGenerator.GNFundamental_DS(model_vars).X_features()

        #Model regression
        model = Regression(X_train,y_train).no_intercept()

        #Model regression train stats
        model_vars['train_pred'] = model.predict(X_train)
        train_ystats,model_vars = train_regression_stats(model,model_vars,X_train,y_train)
        stats = np.append(stats,train_ystats)
        #Predicted ELE: Train data
        model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['train_pred'],
                                                                              model_vars['CWRT'],model_vars['CHWST'])

        #Model power train stats
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)

        #TEST DATA MODEL PREDICTION
        test_vars = test_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        test_vars.columns = ['CWRT','CHWST','Load','ELE']
        
        #Test data features
        y_test = FeatureGenerator.GNFundamental_DS(test_vars).y_label()
        X_test = FeatureGenerator.GNFundamental_DS(test_vars).X_features()
        
        #Model regression stats: Test data
        test_vars['test_pred'] = model.predict(X_test)
        test_ystats,test_vars = test_regression_stats(model,test_vars,X_test,y_test)
        stats = np.append(stats,test_ystats)

        #Power prediction stats: Test data
        test_vars['ELE_pred'] = ModelPower(test_vars).GN_fundamental_model(test_vars['Load'],test_vars['test_pred'],
                                                                              test_vars['CWRT'],test_vars['CHWST'])
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
         #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('GN_DS','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('GN_DS','CV_test')] = test_kWstats[2]

    #Combined stats df
    combined_stats_sets[sample_set] = stats_combiner(combined_stats_sets[sample_set])

# CVratio data for plotting
CVratio_data_GN_DS = CVratio_generator(combined_stats_sets)

CVratio_data_GN_DS['psi'] = psi(CVratio_data_GN_DS,Twb_max,Twb_min)
CVratio_data_GN_DS

**5.5.2 CV barplots: Fundamental GN model with variable DS term**

In [ ]:
CV_barplots(CVratio_data_GN_DS,[0,70],"Fundamental GN model with a variable ${\Delta}S$ term")

**5.5.3 RMSE barplots: Fundamental GN model with variable DS term**

In [ ]:
RMSE_barplots(CVratio_data_GN_DS,[0,100],"Fundamental GN model with a variable ${\Delta}S$ term")

**5.5.4 Monthly variation of test data CV and RMSE: Best Sample**

In [ ]:
best_samples = {'1_month':'sample10','2_months':'sample9','3_months':'sample4','4_months':'sample4'}
stats_GN_DS = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_GN_DS = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    model_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_train = FeatureGenerator.GNFundamental_DS(model_vars).y_label()
    X_train = FeatureGenerator.GNFundamental_DS(model_vars).X_features()

    #Model regression
    model = Regression(X_train, y_train).no_intercept()
    best_models_GN_DS[sample_set] = model

    #y-predictions and model stats:Training data
    model_vars['train_pred'] = model.predict(X_train)

    MAE_ytrain = mean_absolute_error(y_train,model_vars['train_pred'])
    MAE_perc = mean_absolute_percentage_error(y_train,model_vars['train_pred'])
    MBE_ytrain = MAE_ytrain*100/np.mean(y_train)
    MSE_ytrain = mean_squared_error(y_train,model_vars['train_pred'])
    RMSE_ytrain = np.sqrt(MSE_ytrain)
    CV_ytrain = RMSE_ytrain*100/np.mean(y_train)

    #POWER PREDICTION STATS: TRAINING DATA
    model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['train_pred'],
                                                                        model_vars['CWRT'],model_vars['CHWST'])
    
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])


    #y-predictions and model stats: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    test_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_test = FeatureGenerator.GNFundamental_DS(test_vars).y_label()
    X_test = FeatureGenerator.GNFundamental_DS(test_vars).X_features()
    
    test_vars['y'] = y_test
    test_vars['Time_range'] = test_sample_sets[sample_set][best_sample]['Time_range']
    test_vars['month'] = test_sample_sets[sample_set][best_sample]['month']
    test_vars['test_pred'] = model.predict(X_test)

    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        # Model STATS for every test data month
        MAE_ytest = mean_absolute_error(test_vars_month['y'],test_vars_month['test_pred'])
        MBE_ytest = MAE_ytest*100/np.mean(test_vars_month['y'])
        MSE_ytest = mean_squared_error(test_vars_month['y'],test_vars_month['test_pred'])
        RMSE_ytest = np.sqrt(MSE_ytest)
        CV_ytest = RMSE_ytest*100/np.mean(test_vars_month['y'])

        #Power prediction stats: Monthly Test data
        test_vars_month['ELE_pred'] = ModelPower(test_vars_month).GN_fundamental_model(test_vars_month['Load'],test_vars_month['test_pred'],
                                                                        test_vars_month['CWRT'],test_vars_month['CHWST'])
        
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])
    
    stats_GN_DS[sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    #Creating a new column containing the index values
    stats_GN_DS[sample_set] = stats_GN_DS[sample_set].reset_index().rename(columns={'index': 'metering period'})

In [ ]:
selected_sample_CVplots(stats_GN_DS,'Fundamental GN model with variable ${\Delta}S$ term',[0,20])

In [ ]:
selected_sample_RMSEplots(stats_GN_DS,'Fundamental GN model with variable ${\Delta}S$ term',[0,20])

In [ ]:
selected_sample_CV_RMSE_plots(stats_GN_DS,"Fundamental GN model with variable ${\Delta}S$ term",CV_ylim=[0,20],
                              CV_yticks=np.arange(0,20+1,4),RMSE_ylim=[0,20],RMSE_yticks=np.arange(0,20+1,4),
                              ticks_rotation=-30,CVtext_y=0.5,RMSEtext_y=0.5)

**5.5.5 Variation of prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chillerdata).GN_fundamental_DS_model(best_models_GN_DS[sample_set])

# PLOTS
error_variation_plot(error_data,"Fundamental GN model with variable ${\Delta}S$ term")

**5.5.6 Error variation: Overall best sample**

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).GN_fundamental_DS_model(best_models_GN_DS[best_sample_set])

# PLOTS
error_overall_best(error_data,"Fundamental GN model with variable ${\Delta}S$ term",figsize=(10,6),power_ticks=np.arange(0,400+1,40),
                  twb_ticks=np.arange(0,100+1,10))

# **5.6 GN model with intercept (2000)**

**MODEL EQUATION**

$(\frac{1}{COP}+1).\frac{T_{cho}}{T_{cwi}}-1 = a_{0}+a_{1}.\frac{T_{cho}}{Q_{ev}}+a_{2}.\frac{T_{cwi}-T_{cho}}{T_{cwi}.Q_{ev}}+
a_{3}.\frac{Q_{ev}}{T_{cwi}}(1+\frac{1}{COP})$

$y = a_{0}+a_{1}.x_{1}+a_{2}.x_{2}+a_{3}x_{3}$

**5.6.1 Computations**

In [ ]:
combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']

        #Model training features
        y_train = FeatureGenerator.GNFundamental(model_vars).y_label()
        X_train = FeatureGenerator.GNFundamental(model_vars).X_features()

        #Model regression
        model = Regression(X_train,y_train).with_intercept()

        #Model regression train stats
        model_vars['train_pred'] = model.predict(X_train)
        train_ystats,model_vars = train_regression_stats(model,model_vars,X_train,y_train)
        stats = np.append(stats,train_ystats)
        #Predicted ELE: Train data
        model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['train_pred'],
                                                                              model_vars['CWRT'],model_vars['CHWST'])

        #Model power train stats
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)

        #TEST DATA MODEL PREDICTION
        test_vars = test_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        test_vars.columns = ['CWRT','CHWST','Load','ELE']
        
        #Test data features
        y_test = FeatureGenerator.GNFundamental(test_vars).y_label()
        X_test = FeatureGenerator.GNFundamental(test_vars).X_features()
        
        #Model regression stats: Test data
        test_vars['test_pred'] = model.predict(X_test)
        test_ystats,test_vars = test_regression_stats(model,test_vars,X_test,y_test)
        stats = np.append(stats,test_ystats)

        #Power prediction stats: Test data
        test_vars['ELE_pred'] = ModelPower(test_vars).GN_fundamental_model(test_vars['Load'],test_vars['test_pred'],
                                                                              test_vars['CWRT'],test_vars['CHWST'])
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
        #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('GN_w_interc','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('GN_w_interc','CV_test')] = test_kWstats[2]

    #Combined stats df
    combined_stats_sets[sample_set] = stats_combiner(combined_stats_sets[sample_set])
    
# CVratio data for plotting
CVratio_data_GN_w_inter = CVratio_generator(combined_stats_sets)

CVratio_data_GN_w_inter['psi'] = psi(CVratio_data_GN_w_inter,Twb_max,Twb_min)
CVratio_data_GN_w_inter

**5.6.2 CV barplots: Fundamental GN model with intercept**

In [ ]:
CV_barplots(CVratio_data_GN_w_inter,[0,40],"Fundamental GN model with intercept")

**5.6.3 RMSE barplots: Fundamental GN model with intercept**

In [ ]:
RMSE_barplots(CVratio_data_GN_w_inter,[0,60],"Fundamental GN model with intercept")

**5.6.4 Monthly variation of test data CV and RMSE: Best Sample**

In [ ]:
best_samples = {'1_month':'sample10','2_months':'sample9','3_months':'sample4','4_months':'sample4'}
stats_GN_w_interc = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_GN_w_interc = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    model_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_train = FeatureGenerator.GNFundamental(model_vars).y_label()
    X_train = FeatureGenerator.GNFundamental(model_vars).X_features()

    #Model regression
    model = Regression(X_train, y_train).with_intercept()
    best_models_GN_w_interc[sample_set] = model

    #y-predictions and model stats:Training data
    model_vars['train_pred'] = model.predict(X_train)

    MAE_ytrain = mean_absolute_error(y_train,model_vars['train_pred'])
    MAE_perc = mean_absolute_percentage_error(y_train,model_vars['train_pred'])
    MBE_ytrain = MAE_ytrain*100/np.mean(y_train)
    MSE_ytrain = mean_squared_error(y_train,model_vars['train_pred'])
    RMSE_ytrain = np.sqrt(MSE_ytrain)
    CV_ytrain = RMSE_ytrain*100/np.mean(y_train)

    #POWER PREDICTION STATS: TRAINING DATA
    model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['train_pred'],
                                                                              model_vars['CWRT'],model_vars['CHWST'])
    
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])


    #y-predictions and model stats: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    test_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_test = FeatureGenerator.GNFundamental(test_vars).y_label()
    X_test = FeatureGenerator.GNFundamental(test_vars).X_features()
    
    test_vars['y'] = y_test
    test_vars['Time_range'] = test_sample_sets[sample_set][best_sample]['Time_range']
    test_vars['month'] = test_sample_sets[sample_set][best_sample]['month']
    test_vars['test_pred'] = model.predict(X_test)

    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        # Model STATS for every test data month
        MAE_ytest = mean_absolute_error(test_vars_month['y'],test_vars_month['test_pred'])
        MBE_ytest = MAE_ytest*100/np.mean(test_vars_month['y'])
        MSE_ytest = mean_squared_error(test_vars_month['y'],test_vars_month['test_pred'])
        RMSE_ytest = np.sqrt(MSE_ytest)
        CV_ytest = RMSE_ytest*100/np.mean(test_vars_month['y'])

        #Power prediction stats: Monthly Test data
        test_vars_month['ELE_pred'] = ModelPower(test_vars_month).GN_fundamental_model(test_vars_month['Load'],test_vars_month['test_pred'],
                                                                        test_vars_month['CWRT'],test_vars_month['CHWST'])
        
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])
    
    stats_GN_w_interc [sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    #Creating a new column containing the index values
    stats_GN_w_interc [sample_set] = stats_GN_w_interc [sample_set].reset_index().rename(columns={'index': 'metering period'})

In [ ]:
selected_sample_CVplots(stats_GN_w_interc ,'Fundamental GN model with intercept',[0,20])

In [ ]:
selected_sample_RMSEplots(stats_GN_w_interc ,'Fundamental GN model with intercept',[0,20])

In [ ]:
selected_sample_CV_RMSE_plots(stats_GN_w_interc,"Fundamental GN model with intercept",CV_ylim=[0,20],
                              CV_yticks=np.arange(0,20+1,4),RMSE_ylim=[0,20],RMSE_yticks=np.arange(0,20+1,4),
                              ticks_rotation=-30,CVtext_y=0.5,RMSEtext_y=0.5)

**5.6.5 Variation prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chillerdata).GN_fundamental_model(best_models_GN_w_interc[sample_set])

# PLOTS
error_variation_plot(error_data,"Fundamental GN model with intercept")

**5.6.6 Error variation: Overall best sample**

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).GN_fundamental_model(best_models_GN_w_interc[best_sample_set])

# PLOTS
error_overall_best(error_data,"Fundamental GN model with intercept",figsize=(10,6),power_ticks=np.arange(0,400+1,40),
                  twb_ticks=np.arange(0,100+1,10))

# 5.7 GN Model with split Q_leak

* The Fundamental GN model is written in the book 'Cool Thermodynamics (Gordon & Ng (2000)) as:
$\bigg(\frac{1}{COP}+1\bigg).\frac{T_{cho}}{T_{cwi}}-1 = {\Delta}S_{int}.\frac{T_{cho}}{Q_{ev}}+Q_{leak,eqv}.\frac{T_{cwi}-T_{cho}}{T_{cwi}.Q_{ev}}+ R.\frac{Q_{ev}}{T_{cwi}}\bigg(1+\frac{1}{COP}\bigg)$

* The authors assume the heat leakage term as being constant. 

* The Model derivation shows that:
$Q_{leak,eqv} = Q_{leak,evap}+Q_{leak,comp}.\frac{T_{cho}}{T_{cwi}-T_{cho}}$

* This analysis focusses on the effect of **treating the evap and comp leakage terms seperately**.  

* Therefore, the model equation becomes:

$\bigg(\frac{1}{COP}+1\bigg).\frac{T_{cho}}{T_{cwi}}-1 = {\Delta}S_{int}.\frac{T_{cho}}{Q_{ev}}+Q_{leak,evap}.\frac{T_{cwi}-T_{cho}}{T_{cwi}.Q_{ev}}+Q_{leak,comp}.\frac{T_{cho}}{Q_{ev}.T_{cwi}}+ R.\frac{Q_{ev}}{T_{cwi}}\bigg(1+\frac{1}{COP}\bigg)$

* This can be linearized as:
*$y = a_{1}.x_{1}+a_{2}.x_{2}+a_{3}.x_{3}+a_{4}.x_{4}$

**5.7.1 Computations**

In [ ]:
combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']

        #Model training features
        y_train = FeatureGenerator.GNSplit_Qleak(model_vars).y_label()
        X_train = FeatureGenerator.GNSplit_Qleak(model_vars).X_features()

        #Model regression
        model = Regression(X_train,y_train).no_intercept()

        #Model regression train stats
        model_vars['train_pred'] = model.predict(X_train)
        train_ystats,model_vars = train_regression_stats(model,model_vars,X_train,y_train)
        stats = np.append(stats,train_ystats)
        #Predicted ELE: Train data
        model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['train_pred'],
                                                                              model_vars['CWRT'],model_vars['CHWST'])

        #Model power train stats
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)

        #TEST DATA MODEL PREDICTION
        test_vars = test_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        test_vars.columns = ['CWRT','CHWST','Load','ELE']
        
        #Test data features
        y_test = FeatureGenerator.GNSplit_Qleak(test_vars).y_label()
        X_test = FeatureGenerator.GNSplit_Qleak(test_vars).X_features()
        
        #Model regression stats: Test data
        test_vars['test_pred'] = model.predict(X_test)
        test_ystats,test_vars = test_regression_stats(model,test_vars,X_test,y_test)
        stats = np.append(stats,test_ystats)

        #Power prediction stats: Test data
        test_vars['ELE_pred'] = ModelPower(test_vars).GN_fundamental_model(test_vars['Load'],test_vars['test_pred'],
                                                                              test_vars['CWRT'],test_vars['CHWST'])
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
        #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('GN_split','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('GN_split','CV_test')] = test_kWstats[2]

    #Combined stats df
    combined_stats_sets[sample_set] = stats_combiner(combined_stats_sets[sample_set])

# CVratio data for plotting
CVratio_data_GN_split_Qleak = CVratio_generator(combined_stats_sets)

comparison_dfs['1_month']

CVratio_data_GN_split_Qleak['psi'] = psi(CVratio_data_GN_split_Qleak,Twb_max,Twb_min)
CVratio_data_GN_split_Qleak

**5.7.2 CV barplots: Fundamental GN model split Q_leak**

In [ ]:
CV_barplots(CVratio_data_GN_split_Qleak,[0,30],"Fundamental GN model with split $Q_{leak}$")

**5.7.3 RMSE barplots: Fundamental GN model split Q_leak**

In [ ]:
RMSE_barplots(CVratio_data_GN_split_Qleak,[0,60],"Fundamental GN model with split $Q_{leak}$")

**5.7.4 Monthly variation of test data CV and RMSE: Best Sample**

In [ ]:
best_samples = {'1_month':'sample10','2_months':'sample9','3_months':'sample4','4_months':'sample4'}
stats_GN_split = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_GN_split = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    model_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_train = FeatureGenerator.GNSplit_Qleak(model_vars).y_label()
    X_train = FeatureGenerator.GNSplit_Qleak(model_vars).X_features()

    #Model regression
    model = Regression(X_train, y_train).no_intercept()
    best_models_GN_split[sample_set] = model

    #y-predictions and model stats:Training data
    model_vars['train_pred'] = model.predict(X_train)

    MAE_ytrain = mean_absolute_error(y_train,model_vars['train_pred'])
    MAE_perc = mean_absolute_percentage_error(y_train,model_vars['train_pred'])
    MBE_ytrain = MAE_ytrain*100/np.mean(y_train)
    MSE_ytrain = mean_squared_error(y_train,model_vars['train_pred'])
    RMSE_ytrain = np.sqrt(MSE_ytrain)
    CV_ytrain = RMSE_ytrain*100/np.mean(y_train)

    #POWER PREDICTION STATS: TRAINING DATA
    model_vars['ELE_pred'] = ModelPower(model_vars).GN_fundamental_model(model_vars['Load'],model_vars['train_pred'],
                                                                              model_vars['CWRT'],model_vars['CHWST'])
    
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])


    #y-predictions and model stats: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    test_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_test = FeatureGenerator.GNSplit_Qleak(test_vars).y_label()
    X_test = FeatureGenerator.GNSplit_Qleak(test_vars).X_features()
    
    test_vars['y'] = y_test
    test_vars['Time_range'] = test_sample_sets[sample_set][best_sample]['Time_range']
    test_vars['month'] = test_sample_sets[sample_set][best_sample]['month']
    test_vars['test_pred'] = model.predict(X_test)

    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        # Model STATS for every test data month
        MAE_ytest = mean_absolute_error(test_vars_month['y'],test_vars_month['test_pred'])
        MBE_ytest = MAE_ytest*100/np.mean(test_vars_month['y'])
        MSE_ytest = mean_squared_error(test_vars_month['y'],test_vars_month['test_pred'])
        RMSE_ytest = np.sqrt(MSE_ytest)
        CV_ytest = RMSE_ytest*100/np.mean(test_vars_month['y'])

        #Power prediction stats: Monthly Test data
        test_vars_month['ELE_pred'] = ModelPower(test_vars_month).GN_fundamental_model(test_vars_month['Load'],test_vars_month['test_pred'],
                                                                        test_vars_month['CWRT'],test_vars_month['CHWST'])
        
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])

    
    stats_GN_split[sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    #Creating a new column containing the index values
    stats_GN_split[sample_set] = stats_GN_split[sample_set].reset_index().rename(columns={'index': 'metering period'})

In [ ]:
selected_sample_CVplots(stats_GN_split,'Fundamental GN model split Qleak term',[0,20])

In [ ]:
selected_sample_RMSEplots(stats_GN_split,'Fundamental GN model split Qleak term',[0,20])

In [ ]:
selected_sample_CV_RMSE_plots(stats_GN_split,"Fundamental GN model with split $Q_{leak}$",CV_ylim=[0,20],
                              CV_yticks=np.arange(0,20+1,4),RMSE_ylim=[0,20],RMSE_yticks=np.arange(0,20+1,4),
                              ticks_rotation=-30,CVtext_y=0.5,RMSEtext_y=0.5)

**5.7.5 Variation prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chillerdata).GN_split_Qleak_model(best_models_GN_split[sample_set])

# PLOTS
error_variation_plot(error_data,"Fundamental GN model with split Qleak")

**5.7.6 Error variation: Overall best sample**

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).GN_split_Qleak_model(best_models_GN_split[best_sample_set])

# PLOTS
error_overall_best(error_data,"Fundamental GN model with split Qleak",figsize=(10,6),power_ticks=np.arange(0,400+1,40),
                  twb_ticks=np.arange(0,100+1,10))

# 5.8 GN - Foliaco Model (2020)

* B. Foliaco et al. (2020) reformulated the GN fundamental model for easier application to M&V:
* They changed the predicted variable from COP to $\dot{W}$

* Therefore, the model equation becomes:

$W_{ele} - \bigg(\frac{T_{cwi}-T_{cho}}{T_{cho}}\bigg)Q_{ev} = {\Delta}S_{int}T_{cwi}+
Q_{leak,eqv}\bigg(\frac{T_{cwi}-T_{cho}}{T_{cho}}\bigg)+R\bigg(\frac{Q_{ev}^2+Q_{ev}W_{ele}}{T_{cho}}\bigg)$

* This can be linearized as:
*$y = a_{1}.x_{1}+a_{2}.x_{2}+a_{3}.x_{3}$

In [ ]:
combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWST','Load','ELE']

        #Model training features
        y_train = FeatureGenerator.GNFoliaco(model_vars).y_label()
        X_train = FeatureGenerator.GNFoliaco(model_vars).X_features()

        #Model regression
        model = Regression(X_train,y_train).no_intercept()

        #Model regression train stats
        model_vars['train_pred'] = model.predict(X_train)
        train_ystats,model_vars = train_regression_stats(model,model_vars,X_train,y_train)
        stats = np.append(stats,train_ystats)
        #Predicted ELE: Train data
        model_vars['ELE_pred'] = ModelPower(model_vars).GN_Foliaco_model(model_vars['Load'],model_vars['train_pred'],
                                                                              model_vars['CWRT'],model_vars['CHWST'])

        #Model power train stats
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)

        #TEST DATA MODEL PREDICTION
        test_vars = test_sample_sets[sample_set][sample][['CWET','CHLR_LCHWT','Load','ELE']]
        test_vars.columns = ['CWRT','CHWST','Load','ELE']
        
        #Test data features
        y_test = FeatureGenerator.GNFoliaco(test_vars).y_label()
        X_test = FeatureGenerator.GNFoliaco(test_vars).X_features()
        
        #Model regression stats: Test data
        test_vars['test_pred'] = model.predict(X_test)
        test_ystats,test_vars = test_regression_stats(model,test_vars,X_test,y_test)
        stats = np.append(stats,test_ystats)

        #Power prediction stats: Test data
        test_vars['ELE_pred'] = ModelPower(test_vars).GN_Foliaco_model(test_vars['Load'],test_vars['test_pred'],
                                                                              test_vars['CWRT'],test_vars['CHWST'])
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
        #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('GN_Foliaco','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('GN_Foliaco','CV_test')] = test_kWstats[2]

    #Combined stats df
    combined_stats_sets[sample_set] = stats_combiner(combined_stats_sets[sample_set])

# CVratio data for plotting
CVratio_data_GN_Foliaco = CVratio_generator(combined_stats_sets)

CVratio_data_GN_Foliaco['psi'] = psi(CVratio_data_GN_Foliaco,Twb_max,Twb_min)
CVratio_data_GN_Foliaco

**5.8.2 CV barplots: Foliaco reformulated GN Model**

In [ ]:
CV_barplots(CVratio_data_GN_Foliaco,[0,30],"Fundamental Foliaco reformulated GN model")

**5.8.3 RMSE barplots: Folicaco reformulated GN model**

In [ ]:
RMSE_barplots(CVratio_data_GN_Foliaco,[0,60],"Foliaco reformulated GN model")

**5.8.4 Monthly variation of test data CV and RMSE: Best Sample**

In [ ]:
best_samples = {'1_month':'sample10','2_months':'sample4','3_months':'sample4','4_months':'sample3'}
stats_GN_Foliaco = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_GN_Foliaco = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    model_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_train = FeatureGenerator.GNFoliaco(model_vars).y_label()
    X_train = FeatureGenerator.GNFoliaco(model_vars).X_features()

    #Model regression
    model = Regression(X_train, y_train).no_intercept()
    best_models_GN_Foliaco[sample_set] = model

    #y-predictions and model stats:Training data
    model_vars['train_pred'] = model.predict(X_train)

    MAE_ytrain = mean_absolute_error(y_train,model_vars['train_pred'])
    MAE_perc = mean_absolute_percentage_error(y_train,model_vars['train_pred'])
    MBE_ytrain = MAE_ytrain*100/np.mean(y_train)
    MSE_ytrain = mean_squared_error(y_train,model_vars['train_pred'])
    RMSE_ytrain = np.sqrt(MSE_ytrain)
    CV_ytrain = RMSE_ytrain*100/np.mean(y_train)

    #POWER PREDICTION STATS: TRAINING DATA
    model_vars['ELE_pred'] = ModelPower(model_vars).GN_Foliaco_model(model_vars['Load'],model_vars['train_pred'],
                                                                              model_vars['CWRT'],model_vars['CHWST'])
    
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])


    #y-predictions and model stats: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['CWET','CHLR_LCHWT','Load','ELE']]
    test_vars.columns = ['CWRT','CHWST','Load','ELE']
    
    #Extract only the features to be used for regression
    y_test = FeatureGenerator.GNFoliaco(test_vars).y_label()
    X_test = FeatureGenerator.GNFoliaco(test_vars).X_features()
    
    test_vars['y'] = y_test
    test_vars['Time_range'] = test_sample_sets[sample_set][best_sample]['Time_range']
    test_vars['month'] = test_sample_sets[sample_set][best_sample]['month']
    test_vars['test_pred'] = model.predict(X_test)

    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        # Model STATS for every test data month
        MAE_ytest = mean_absolute_error(test_vars_month['y'],test_vars_month['test_pred'])
        MBE_ytest = MAE_ytest*100/np.mean(test_vars_month['y'])
        MSE_ytest = mean_squared_error(test_vars_month['y'],test_vars_month['test_pred'])
        RMSE_ytest = np.sqrt(MSE_ytest)
        CV_ytest = RMSE_ytest*100/np.mean(test_vars_month['y'])

        #Power prediction stats: Monthly Test data
        test_vars_month['ELE_pred'] = ModelPower(test_vars_month).GN_Foliaco_model(test_vars_month['Load'],test_vars_month['test_pred'],
                                                                        test_vars_month['CWRT'],test_vars_month['CHWST'])
        
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])

    
    stats_GN_Foliaco[sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    #Creating a new column containing the index values
    stats_GN_Foliaco[sample_set] = stats_GN_Foliaco[sample_set].reset_index().rename(columns={'index': 'metering period'})
    
stats_GN_Foliaco['4_months']

In [ ]:
selected_sample_CVplots(performance_stats_cell=stats_GN_Foliaco,model_title='Foliaco reformulated GN model',ylim=[0,30])

In [ ]:
selected_sample_RMSEplots(stats_GN_Foliaco,model_title='Foliaco reformulated GN model',ylim=[0,20])

In [ ]:
selected_sample_CV_RMSE_plots(stats_GN_Foliaco,"Foliaco reformulated GN model",CV_ylim=[0,32],
                              CV_yticks=np.arange(0,32+1,4),RMSE_ylim=[0,32],RMSE_yticks=np.arange(0,32+1,4),
                              ticks_rotation=-30,CVtext_y=0.5,RMSEtext_y=0.5)

**5.8.5 Variation prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chillerdata).GN_Foliaco_model(best_models_GN_Foliaco[sample_set])

# PLOTS
error_variation_plot(error_data,"Foliaco reformulated GN model")

**5.8.6 Error variation: Overall best sample**

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).GN_Foliaco_model(best_models_GN_Foliaco[best_sample_set])

#PLOTS
error_overall_best(error_data,"Reformulated Foliaco GN model",figsize=(10,6),power_ticks=np.arange(0,400+1,40),
                  twb_ticks=np.arange(0,100+1,10))

# **5.9 Bi-quadratic black-box model**

**MODEL EQUATION**

$\frac{1}{COP}=b_{0}+b_{1}.\frac{1}{Q_{ev}}+b_{2}.Q_{ev}+
b_{3}.\frac{T_{cwi}}{Q_{ev}}+b_{4}.\frac{T_{cwi}^{2}}{Q_{ev}}+b_{5}.T_{cwi}+b_{6}.Q_{ev}T_{cwi}+b_{7}.T_{cwi}^{2}+b_{8}.Q_{ev}T_{cwi}^2$

$y = b_{0}+b_{1}.x_{1}+b_{2}.x_{2}+b_{3}x_{3}+b_{4}x_{4}+b_{5}x_{5}
+b_{6}x_{6}+b_{7}x_{7}+b_{8}x_{8}$

**5.9.1 Computations**

In [ ]:
combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['CWET','Load','ELE']]
        model_vars.columns = ['CWRT','Load','ELE']

        #Model training features
        y_train = FeatureGenerator.BiQuadratic(model_vars).y_label()
        X_train = FeatureGenerator.BiQuadratic(model_vars).X_features()

        #Model regression
        model = Regression(X_train,y_train).with_intercept()

        #Model regression train stats
        model_vars['train_pred'] = model.predict(X_train)
        train_ystats,model_vars = train_regression_stats(model,model_vars,X_train,y_train)
        stats = np.append(stats,train_ystats)
        #Predicted ELE: Train data
        model_vars['ELE_pred'] = ModelPower(model_vars).biquadratic_model(model_vars['Load'],model_vars['train_pred'])

        #Model power train stats
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)

        #TEST DATA MODEL PREDICTION
        test_vars = test_sample_sets[sample_set][sample][['CWET','Load','ELE']]
        test_vars.columns = ['CWRT','Load','ELE']
        
        #Test data features
        y_test = FeatureGenerator.BiQuadratic(test_vars).y_label()
        X_test = FeatureGenerator.BiQuadratic(test_vars).X_features()
        
        #Model regression stats: Test data
        test_vars['test_pred'] = model.predict(X_test)
        test_ystats,test_vars = test_regression_stats(model,test_vars,X_test,y_test)
        stats = np.append(stats,test_ystats)

        #Power prediction stats: Test data
        test_vars['ELE_pred'] = ModelPower(test_vars).biquadratic_model(test_vars['Load'],test_vars['test_pred'])
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
        #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('biquadratic','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('biquadratic','CV_test')] = test_kWstats[2]

    #Combined stats df
    combined_stats_sets[sample_set] = stats_combiner(combined_stats_sets[sample_set])

# CVratio data for plotting
CVratio_data_biquadratic = CVratio_generator(combined_stats_sets)

CVratio_data_biquadratic['psi'] = psi(CVratio_data_biquadratic,Twb_max,Twb_min)
CVratio_data_biquadratic

**5.9.2 CV barplots: Bi-quadratic black-box model**

In [ ]:
CV_fig = CV_barplots(CVratio_data_biquadratic,[0,100],"Biquadratic model")
CV_fig.savefig('CH1_Performance_deterioration_biquadratic.png',bbox_inches='tight')

**5.9.3 RMSE barplots: Bi-quadratic black-box model**

In [ ]:
RMSE_barplots(CVratio_data_biquadratic,[0,100],"Biquadratic model")

**5.9.4 Monthly variation of test data CV and RMSE: Best Sample**

In [ ]:
best_samples = {'1_month':'sample10','2_months':'sample9','3_months':'sample8','4_months':'sample3'}
stats_biquadratic = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_biquadratic = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['CWET','Load','ELE']]
    model_vars.columns = ['CWRT','Load','ELE']
    
    #Extract only the features to be used for regression
    y_train = FeatureGenerator.BiQuadratic(model_vars).y_label()
    X_train = FeatureGenerator.BiQuadratic(model_vars).X_features()

    #Model regression
    model = Regression(X_train, y_train).with_intercept()
    best_models_biquadratic[sample_set] = model

    #y-predictions and model stats:Training data
    model_vars['train_pred'] = model.predict(X_train)

    MAE_ytrain = mean_absolute_error(y_train,model_vars['train_pred'])
    MAE_perc = mean_absolute_percentage_error(y_train,model_vars['train_pred'])
    MBE_ytrain = MAE_ytrain*100/np.mean(y_train)
    MSE_ytrain = mean_squared_error(y_train,model_vars['train_pred'])
    RMSE_ytrain = np.sqrt(MSE_ytrain)
    CV_ytrain = RMSE_ytrain*100/np.mean(y_train)

    #POWER PREDICTION STATS: TRAINING DATA
    model_vars['ELE_pred'] = ModelPower(model_vars).biquadratic_model(model_vars['Load'],model_vars['train_pred'])
    
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])


    #y-predictions and model stats: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['CWET','Load','ELE']]
    test_vars.columns = ['CWRT','Load','ELE']
    
    #Extract only the features to be used for regression
    y_test = FeatureGenerator.BiQuadratic(test_vars).y_label()
    X_test = FeatureGenerator.BiQuadratic(test_vars).X_features()
    
    test_vars['y'] = y_test
    test_vars['Time_range'] = test_sample_sets[sample_set][best_sample]['Time_range']
    test_vars['month'] = test_sample_sets[sample_set][best_sample]['month']
    test_vars['test_pred'] = model.predict(X_test)

    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        # Model STATS for every test data month
        MAE_ytest = mean_absolute_error(test_vars_month['y'],test_vars_month['test_pred'])
        MBE_ytest = MAE_ytest*100/np.mean(test_vars_month['y'])
        MSE_ytest = mean_squared_error(test_vars_month['y'],test_vars_month['test_pred'])
        RMSE_ytest = np.sqrt(MSE_ytest)
        CV_ytest = RMSE_ytest*100/np.mean(test_vars_month['y'])

        #Power prediction stats: Monthly Test data
        test_vars_month['ELE_pred'] = ModelPower(test_vars_month).biquadratic_model(test_vars_month['Load'],test_vars_month['test_pred'])
        
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])

    
    stats_biquadratic[sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    #Creating a new column containing the index values
    stats_biquadratic[sample_set] = stats_biquadratic[sample_set].reset_index().rename(columns={'index': 'metering period'})
    
stats_biquadratic['4_months']

In [ ]:
selected_sample_CVplots(stats_biquadratic,'Biquadratic model',[0,16])

In [ ]:
selected_sample_RMSEplots(stats_biquadratic,'Biquadratic model',[0,25])

In [ ]:
fig_CV_RMSE = selected_sample_CV_RMSE_plots(stats_biquadratic,model_title="",CV_ylim=[0,20],
                              CV_yticks=np.arange(0,20+1,4),RMSE_ylim=[0,25],RMSE_yticks=np.arange(0,25+1,5),
                              ticks_rotation=-30,CVtext_y=0.25,RMSEtext_y=0.25,CVtext_x=2.5,RMSEtext_x=2.5,title_y=-4)

fig_CV_RMSE.subplots_adjust(hspace=.25,wspace=0.01)

fig_CV_RMSE.savefig('CH1_Best_samples_biquadratic.png',bbox_inches='tight')

**5.9.5 Variation prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chillerdata).biquadratic_model(best_models_biquadratic[sample_set])
     
# PLOTS
error_variation_plot(error_data,"Bi-quadratic model")

**5.9.6 Error variation: Overall best sample**

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).biquadratic_model(best_models_biquadratic[best_sample_set])

# PLOTS
error_overall_best(error_data,"Bi-quadratic model",figsize=(10,6))

# **5.10 Multi-variate (2-degree Polynomial) black box model**

**MODEL EQUATION**

$COP=b_{0}+b_{1}.Q_{ev}+b_{2}.T_{chi}+b_{3}.T_{cwi}+ b_{4}.Q_{ev}^{2}+b_{5}.T_{chi}^{2}+b_{6}.T_{cwi}^{2}+b_{7}.Q_{ev}T_{chi}+b_{8}.Q_{ev}T_{cwi}+b_{9}.T_{chi}T_{cwi}$

$y = b_{0}+b_{1}.x_{1}+b_{2}.x_{2}+b_{3}x_{3}+b_{4}x_{4}+b_{5}x_{5}
+b_{6}x_{6}+b_{7}x_{7}+b_{8}x_{8}+b_{9}x_{9}$

**5.10.1 Calculations**

In [ ]:
combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['CWET','CHLR_RCHWT','Load','ELE']]
        model_vars.columns = ['CWRT','CHWRT','Load','ELE']

        #Model training features
        y_train = FeatureGenerator.MultiVariate(model_vars).y_label()
        X_train = FeatureGenerator.MultiVariate(model_vars).X_features()

        #Model regression
        model = Regression(X_train,y_train).with_intercept()

        #Model regression train stats
        model_vars['train_pred'] = model.predict(X_train)
        train_ystats,model_vars = train_regression_stats(model,model_vars,X_train,y_train)
        stats = np.append(stats,train_ystats)
        #Predicted ELE: Train data
        model_vars['ELE_pred'] = ModelPower(model_vars).multivariate_model(model_vars['Load'],model_vars['train_pred'])

        #Model power train stats
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)

        #TEST DATA MODEL PREDICTION
        test_vars = test_sample_sets[sample_set][sample][['CWET','CHLR_RCHWT','Load','ELE']]
        test_vars.columns = ['CWRT','CHWRT','Load','ELE']
        
        #Test data features
        y_test = FeatureGenerator.MultiVariate(test_vars).y_label()
        X_test = FeatureGenerator.MultiVariate(test_vars).X_features()
        
        #Model regression stats: Test data
        test_vars['test_pred'] = model.predict(X_test)
        test_ystats,test_vars = test_regression_stats(model,test_vars,X_test,y_test)
        stats = np.append(stats,test_ystats)

        #Power prediction stats: Test data
        test_vars['ELE_pred'] = ModelPower(test_vars).multivariate_model(test_vars['Load'],test_vars['test_pred'])
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
        #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('multivariate','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('multivariate','CV_test')] = test_kWstats[2]

    #Combined stats df
    combined_stats_sets[sample_set] = stats_combiner(combined_stats_sets[sample_set])

# CVratio data for plotting
CVratio_data_multivariate = CVratio_generator(combined_stats_sets)

CVratio_data_multivariate['psi'] = psi(CVratio_data_multivariate,Twb_max,Twb_min)
CVratio_data_multivariate

**5.10.2 CV barplots: Multi-variate polynomial model**

In [ ]:
CV_barplots(CVratio_data_multivariate,[0,100],"Multivariate polynomial model")

**5.10.3 RMSE barplots: Multi-variate polynomial model**

In [ ]:
RMSE_barplots(CVratio_data_multivariate,[0,120],"Multivariate polynomial model")

**5.10.4 Monthly variation of test data CV and RMSE: Best Samples**

In [ ]:
best_samples = {'1_month':'sample6','2_months':'sample4','3_months':'sample8','4_months':'sample7'}
stats_multivariate = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_multivariate = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}


for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['CWET','CHLR_RCHWT','Load','ELE']]
    model_vars.columns = ['CWRT','CHWRT','Load','ELE']
   
    #Extract only the features to be used for regression
    y_train = FeatureGenerator.MultiVariate(model_vars).y_label()
    X_train = FeatureGenerator.MultiVariate(model_vars).X_features()

    #Model regression
    model = Regression(X_train, y_train).with_intercept()
    best_models_multivariate[sample_set] = model

    #y-predictions and model stats:Training data
    model_vars['train_pred'] = model.predict(X_train)

    MAE_ytrain = mean_absolute_error(y_train,model_vars['train_pred'])
    MAE_perc = mean_absolute_percentage_error(y_train,model_vars['train_pred'])
    MBE_ytrain = MAE_ytrain*100/np.mean(y_train)
    MSE_ytrain = mean_squared_error(y_train,model_vars['train_pred'])
    RMSE_ytrain = np.sqrt(MSE_ytrain)
    CV_ytrain = RMSE_ytrain*100/np.mean(y_train)

    #POWER PREDICTION STATS: TRAINING DATA
    model_vars['ELE_pred'] = ModelPower(model_vars).multivariate_model(model_vars['Load'],model_vars['train_pred'])
    
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])


    #y-predictions and model stats: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['CWET','CHLR_RCHWT','Load','ELE']]
    test_vars.columns = ['CWRT','CHWRT','Load','ELE']
    
    #Extract only the features to be used for regression
    y_test = FeatureGenerator.MultiVariate(test_vars).y_label()
    X_test = FeatureGenerator.MultiVariate(test_vars).X_features()
    
    test_vars['y'] = y_test
    test_vars['Time_range'] = test_sample_sets[sample_set][best_sample]['Time_range']
    test_vars['month'] = test_sample_sets[sample_set][best_sample]['month']
    test_vars['test_pred'] = model.predict(X_test)

    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        # Model STATS for every test data month
        MAE_ytest = mean_absolute_error(test_vars_month['y'],test_vars_month['test_pred'])
        MBE_ytest = MAE_ytest*100/np.mean(test_vars_month['y'])
        MSE_ytest = mean_squared_error(test_vars_month['y'],test_vars_month['test_pred'])
        RMSE_ytest = np.sqrt(MSE_ytest)
        CV_ytest = RMSE_ytest*100/np.mean(test_vars_month['y'])

        #Power prediction stats: Monthly Test data
        test_vars_month['ELE_pred'] = ModelPower(test_vars_month).multivariate_model(test_vars_month['Load'],
                                                                                     test_vars_month['test_pred'])
        
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])

    
    stats_multivariate[sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    #Creating a new column containing the index values
    stats_multivariate[sample_set] = stats_multivariate[sample_set].reset_index().rename(columns={'index': 'metering period'})
    
stats_multivariate['2_months']

In [ ]:
selected_sample_CVplots(stats_multivariate,'Multivariate polynomial model',[0,25])

In [ ]:
selected_sample_RMSEplots(stats_multivariate,'Multivariate polynomial model',[0,30])

In [ ]:
selected_sample_CV_RMSE_plots(stats_multivariate,"Multivariate polynomial model",CV_ylim=[0,25],
                              CV_yticks=np.arange(0,25+1,5),RMSE_ylim=[0,25],RMSE_yticks=np.arange(0,25+1,5),
                              ticks_rotation=-30,CVtext_y=0.5,RMSEtext_y=0.5)

**5.10.5 Variation prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chillerdata).multivariate_model(best_models_multivariate[sample_set])
     
# PLOTS
error_variation_plot(error_data,"Multivariate polynomial model")

**5.10.6 Error variation: Overall best sample**

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).multivariate_model(best_models_multivariate[best_sample_set])

# PLOTS
error_overall_best(error_data,"Multivariate polynomial model",figsize=(10,6),power_ticks=np.arange(0,400+1,40),
                  twb_ticks=np.arange(0,100+1,10))

# **5.11 CP power model: A.Q. Mohammad et. al (2017)**

**MODEL EQUATION**

$ELE = b_{1}(T_{wb}-b_{2})^{-}+b_{3}(T_{wb}-b_{2})^{+}+b_{4}$

**5.11.1 Calculations**

In [ ]:
from scipy.optimize import curve_fit

#Model function
def four_CP(x,b1,b2,b3,b4):
    return np.piecewise(x, [x < b2], [lambda x:b1*x + b4-b1*b2, lambda x:b3*x + b4-b3*b2])

combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['Twb','ELE','Load']]
        model_vars['1/COP'] = model_vars['ELE']/model_vars['Load']
        model_vars['NewIndex'] = range(len(model_vars))
        model_vars.set_index('NewIndex',inplace=True,drop=True)
        X = model_vars['Twb']
        y = model_vars['ELE']

        #Initial guesses
        CP = (X.min()+X.max())/2
        y_Xmax = y.iloc[X.idxmax()]
        y_Xmin = y.iloc[X.idxmin()]
        Ycp = (y_Xmax+y_Xmin)/2
        LS = (Ycp-y_Xmin)/(CP-X.min())
        RS = (y_Xmax-Ycp)/(X.max()-CP)      
        initialGuess = [LS,CP,RS,Ycp]

        #Perform the curve-fit
        xdata = X.values
        ydata = y.values
        coeffs,covariance = curve_fit(four_CP,xdata,ydata,initialGuess)

        #Model power train stats
        model_vars['ELE_pred'] = four_CP(model_vars['Twb'].to_numpy(),coeffs[0],coeffs[1],coeffs[2],coeffs[3])
        model_vars['1/COP_pred'] = model_vars['ELE_pred']/model_vars['Load']
        model_vars['COP_pred'] = model_vars['Load']/model_vars['ELE_pred']
        model_vars['res^2'] = (model_vars['ELE']-model_vars['ELE_pred'])**2
        model_vars['var'] = (model_vars['ELE']-model_vars['ELE'].mean())**2

        R2 = 1-model_vars['res^2'].sum()/model_vars['var'].sum()
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)
    
        #model stats: Test data
        test_vars = test_sample_sets[sample_set][sample][['Twb','ELE','Load']]
        test_vars['COP'] = test_vars['Load']/test_vars['ELE']
    
        #Model prediction with the test data
        test_vars['ELE_pred'] = four_CP(test_vars['Twb'].to_numpy(),coeffs[0],coeffs[1],coeffs[2],coeffs[3])
        
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        
         #Fill the stats and combined stats arrays
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
        #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('4CP','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('4CP','CV_test')] = test_kWstats[2]
    
    #Combined stats df
    combined_stats_sets[sample_set] = pd.DataFrame(combined_stats_sets[sample_set].reshape((12,6)), index=samples[sample_set], 
                              columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain','MBE_kWtest','RMSE_kWtest','CV_kWtest'])

    combined_stats_sets[sample_set]['CV_ratio']  = ['-']*12
    combined_stats_sets[sample_set]['CV_kW_ratio'] = combined_stats_sets[sample_set]['CV_kWtest']/combined_stats_sets[sample_set]['CV_kWtrain']
    combined_stats_sets[sample_set]['CV_kW_net'] = 0.5*((combined_stats_sets[sample_set]['CV_kWtest']+combined_stats_sets[sample_set]['CV_kWtrain'])/2+
                                                        np.sqrt(combined_stats_sets[sample_set]['CV_kWtest']*combined_stats_sets[sample_set]['CV_kWtrain']))*(combined_stats_sets[sample_set]['CV_kW_ratio'])**0.25

# CVratio data for plotting
CVratio_data_4CP = CVratio_generator(combined_stats_sets)

CVratio_data_4CP['psi'] = psi(CVratio_data_4CP,Twb_max,Twb_min)
CVratio_data_4CP

**5.11.2  CV barplots: 4CP model**

In [ ]:
CV_barplots(CVratio_data_4CP,[0,50],"4P CP model")

**5.11.3 RMSE barplots: 4CP model**

In [ ]:
RMSE_barplots(CVratio_data_4CP,[0,100],"4P CP model")                                                                                                                                                                                                                                                                                                                                                                                                                                        

**5.11.4 Monthly variation of test data CV and RMSE: Best Samples**

In [ ]:
best_samples = {'1_month':'sample9','2_months':'sample7','3_months':'sample9','4_months':'sample8'}
stats_4CP = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_4CP = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['Twb','ELE','Load']]
    #Extract only the features to be used for regression
    model_vars['1/COP'] = model_vars['ELE']/model_vars['Load']
    model_vars['NewIndex'] = range(len(model_vars))
    model_vars.set_index('NewIndex',inplace=True,drop=True)
    X = model_vars['Twb']
    y = model_vars['ELE']

    #Initial guesses
    CP = (X.min()+X.max())/2
    y_Xmax = y.iloc[X.idxmax()]
    y_Xmin = y.iloc[X.idxmin()]
    Ycp = (y_Xmax+y_Xmin)/2
    LS = (Ycp-y_Xmin)/(CP-X.min())
    RS = (y_Xmax-Ycp)/(X.max()-CP)      
    initialGuess = [LS,CP,RS,Ycp]

   #Perform the curve-fit
    xdata = X.values
    ydata = y.values
    coeffs,covariance = curve_fit(four_CP,xdata,ydata,initialGuess) #The function elec is defined in section 4.8.1
    best_models_4CP[sample_set] = coeffs  #To be used in the proceeding computations

    #Model power train stats
    model_vars['ELE_pred'] = four_CP(model_vars['Twb'].to_numpy(),coeffs[0],coeffs[1],coeffs[2],coeffs[3])
    model_vars['COP_pred'] = model_vars['Load']/model_vars['ELE_pred']
    model_vars['res^2'] = (model_vars['ELE']-model_vars['ELE_pred'])**2
    model_vars['var'] = (model_vars['ELE']-model_vars['ELE'].mean())**2

    R2 = 1-model_vars['res^2'].sum()/model_vars['var'].sum()
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])

    #Model predictions: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['Twb','ELE','Load']]
    
    test_vars['COP'] = test_vars['Load']/test_vars['ELE']
    test_vars['Time_range'] = test_sample_sets[sample_set][best_sample]['Time_range']
    test_vars['month'] = test_sample_sets[sample_set][best_sample]['month']

    #Model prediction with the test data
    test_vars['ELE_pred'] = four_CP(test_vars['Twb'].to_numpy(),coeffs[0],coeffs[1],coeffs[2],coeffs[3])

    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        #Power prediction stats: Monthly Test data
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])
    
    stats_4CP[sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    #Creating a new column containing the index values
    stats_4CP[sample_set] = stats_4CP[sample_set].reset_index().rename(columns={'index': 'metering period'})

In [ ]:
selected_sample_CVplots(stats_4CP,'4P CP model',[0,60])

In [ ]:
selected_sample_RMSEplots(stats_4CP,'4CP model',[0,80])

In [ ]:
selected_sample_CV_RMSE_plots(stats_4CP,"4P CP model",CV_ylim=[0,60],
                              CV_yticks=np.arange(0,60+1,15),RMSE_ylim=[0,60],RMSE_yticks=np.arange(0,60+1,15),
                              ticks_rotation=-30,CVtext_y=1,RMSEtext_y=1)

**5.11.5 Variation prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chillerdata).CP_model(best_models_4CP[sample_set])
     
# PLOTS
error_variation_plot(error_data,"4P CP model")

**5.11.6 Error variation: Overall best sample**

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).CP_model(best_models_4CP[best_sample_set])

# PLOTS
error_overall_best(error_data,"4P CP model",figsize=(10,6))

# **5.12 Reference curves method**

In [ ]:
#Chiller Parameters
Q_rated = 2110 #kW
ELE_rated = 341.4 #kW
COP_rated = Q_rated/ELE_rated
CWRT_rated = (84-32)*5/9
CHWST_rated = (44-32)*5/9

#CANDIDATE CURVES
CAPFT = [[0.5359238,0.01785857,0.001579272,0.04522868,-0.001632172,0.0006008061],
        [0.6322235,0.01553954,-0.00914977,0.05284289,-0.002818835,0.007074096],
        [0.5474968,-0.004800356,-0.006845886,0.06302552,-0.002911528,0.00659869],
        [0.7585741,-0.05127829,0.005848602,0.04235544,-0.001606561,0.0009364594],
        [0.7734286,0.04848829,-0.001859209,0.02611971,-0.001243562,0.0007210181]]

EIRFT = [[0.6270855,-0.04119654,0.001327107,0.02661446,-0.0001856827,0.0005094942],
        [0.910673,-0.00378641,-0.001084261,-0.007148246,0.0005178799,0.00001818623],
        [0.8234137,0.01308264,-0.001147806,-0.004071161,0.0005220869,-0.0006312495],
        [0.7563409,-0.01966623,-0.0003927494,0.02973607,0.000129923,-0.0004325357],
        [0.7203736,0.0103046,-0.0007854237,-0.001206349,0.000821006,-0.001455414]]

EIRFPLR = [[0.2469187,0.186269,0.5660904],[0.2595694,0.1889684,0.551845],[0.2653493,0.144462,0.5911158],
          [0.06917141,0.1743339,0.7524059],[0.08653354,0.1737382,0.7403679]]

combined_stats_sets = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set  in combined_stats_sets.keys():
    for sample in metering_sample_sets[sample_set]:
        stats = []
        model_vars = metering_sample_sets[sample_set][sample][['Time_range','CHLR_LCHWT','CWET','Load','ELE']]
        model_vars.columns = ['Time_range','CHWST','CWRT','Load','ELE']

        #Units conversion to kW and C
        model_vars['CHWST'] = (model_vars['CHWST']-32)*5/9
        model_vars['CWRT'] = (model_vars['CWRT']-32)*5/9

        curve_vars_dfs = []
        CV_star = []
        Qref_values = []
        Pref_values = []
        for i in range(len(CAPFT)):
            curve_vars = model_vars.copy()
            #Chiller Curve functions
            def capft(CHWST,CWRT,a0=CAPFT[i][0],a1=CAPFT[i][1],a2=CAPFT[i][2],a3=CAPFT[i][3],a4=CAPFT[i][4],a5=CAPFT[i][5]):
                return a0+a1*CHWST+a2*CHWST**2+a3*CWRT+a4*CWRT**2+a5*CHWST*CWRT

            def eirft(CHWST,CWRT,b0=EIRFT[i][0],b1=EIRFT[i][1],b2=EIRFT[i][2],b3=EIRFT[i][3],b4=EIRFT[i][4],b5=EIRFT[i][5]):
                return b0+b1*CHWST+b2*CHWST**2+b3*CWRT+b4*CWRT**2+b5*CHWST*CWRT

            def eirfplr(PLR,c0=EIRFPLR[i][0],c1=EIRFPLR[i][1],c2=EIRFPLR[i][2]):
                return c0+c1*PLR+c2*PLR**2
        
            #Curve computations
            curve_vars['CAPFT']=np.vectorize(capft)(curve_vars['CHWST'],curve_vars['CWRT'])
            curve_vars['EIRFT']=np.vectorize(eirft)(curve_vars['CHWST'],curve_vars['CWRT'])
            CAPFT_rated = capft(CHWST_rated,CWRT_rated)
            Qref = Q_rated/CAPFT_rated
            curve_vars['PLR']= curve_vars['Load']/(Qref*curve_vars['CAPFT'])
            curve_vars['EIRFPLR'] = np.vectorize(eirfplr)(curve_vars['PLR'])
            curve_vars['K'] = curve_vars['CAPFT']*curve_vars['EIRFT']*curve_vars['EIRFPLR']/curve_vars['ELE']
            Pref = len(curve_vars)/curve_vars['K'].sum()
            curve_vars['ELE_pred'] = curve_vars['CAPFT']*curve_vars['EIRFT']*curve_vars['EIRFPLR']*Pref
            curve_vars['sq_res_ratio'] = ((curve_vars['ELE']-curve_vars['ELE_pred'])/curve_vars['ELE'])**2 
    
            curve_vars_dfs.append(curve_vars)
            Qref_values = np.append(Qref_values,Qref)
            Pref_values = np.append(Pref_values,Pref)
            CV_star = np.append(CV_star,np.sqrt(curve_vars['sq_res_ratio'].sum()/len(curve_vars))*100)

        #Choice of the best curve for the chiller
        CV_best = CV_star.min()
        CV_best_posn = CV_star.argmin()
        Qref_best = Qref_values[CV_best_posn]
        Pref_best = Pref_values[CV_best_posn]
        CAPFT_best = CAPFT[CV_best_posn]
        EIRFT_best = EIRFT[CV_best_posn]
        EIRFPLR_best = EIRFPLR[CV_best_posn]

        model_vars = curve_vars_dfs[CV_best_posn]
        model_vars['COP'] = model_vars['Load']/model_vars['ELE']
        model_vars['1/COP'] = 1/model_vars['COP']
        model_vars['1/COP_pred'] = model_vars['ELE_pred']/model_vars['Load']

        #Power prediction statistics
        model_vars['res^2'] = (model_vars['ELE']-model_vars['ELE_pred'])**2
        model_vars['var'] = (model_vars['ELE']-model_vars['ELE'].mean())**2

        R2 = 1-model_vars['res^2'].sum()/model_vars['var'].sum()
        train_kWstats = train_power_prediction_stats(model_vars)
        stats = np.append(stats,train_kWstats)    
        
        #model stats: Test data
        test_vars = test_sample_sets[sample_set][sample][['Time_range','CHLR_LCHWT','CWET','Load','ELE']]
        test_vars.columns = ['Time_range','CHWST','CWRT','Load','ELE']

        #Units conversion to kW and C
        test_vars['CHWST'] = (test_vars['CHWST']-32)*5/9
        test_vars['CWRT'] = (test_vars['CWRT']-32)*5/9

        test_vars['COP'] = test_vars['Load']/test_vars['ELE']
        test_vars['1/COP'] = 1/test_vars['COP']
    
        #Model prediction with the test data
        test_vars['CAPFT'] = capft(test_vars['CHWST'],test_vars['CWRT'],CAPFT_best[0],CAPFT_best[1],CAPFT_best[2],
                                   CAPFT_best[3],CAPFT_best[4],CAPFT_best[5])
        test_vars['EIRFT'] = eirft(test_vars['CHWST'],test_vars['CWRT'],EIRFT_best[0],EIRFT_best[1],EIRFT_best[2],
                                   EIRFT_best[3],EIRFT_best[4],EIRFT_best[5])
        test_vars['PLR'] = test_vars['Load']/(Qref_best*test_vars['CAPFT'])
        test_vars['EIRFPLR'] = eirfplr(test_vars['PLR'],EIRFPLR_best[0],EIRFPLR_best[1],EIRFPLR_best[2])
        #Power prediction stats: Test data
        test_vars['ELE_pred'] = Pref_best*test_vars['CAPFT']*test_vars['EIRFT']*test_vars['EIRFPLR']
        test_vars['COP_pred'] = test_vars['Load']/test_vars['ELE_pred']
        test_vars['1/COP_pred'] = 1/test_vars['COP_pred']
        
        test_kWstats = test_power_prediction_stats(test_vars)
        stats = np.append(stats,test_kWstats)
        
        #Fill the stats and combined stats arrays
        combined_stats_sets[sample_set] = np.append(combined_stats_sets[sample_set],stats)
        
        #Feed the DataFrame for comparison of all the models
        comparison_dfs[sample_set].loc[sample,('RefCurves','CV_train')] = train_kWstats[2]
        comparison_dfs[sample_set].loc[sample,('RefCurves','CV_test')] = test_kWstats[2]
    
    #Combined stats df
    combined_stats_sets[sample_set] = pd.DataFrame(combined_stats_sets[sample_set].reshape((12,6)), index=samples[sample_set], 
                              columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain','MBE_kWtest','RMSE_kWtest','CV_kWtest'])


    combined_stats_sets[sample_set]['CV_ratio'] = ['-']*12
    combined_stats_sets[sample_set]['CV_kW_ratio'] = combined_stats_sets[sample_set]['CV_kWtest']/combined_stats_sets[sample_set]['CV_kWtrain']
    combined_stats_sets[sample_set]['CV_kW_net'] = 0.5*((combined_stats_sets[sample_set]['CV_kWtest']+combined_stats_sets[sample_set]['CV_kWtrain'])/2+
                                                        np.sqrt(combined_stats_sets[sample_set]['CV_kWtest']*combined_stats_sets[sample_set]['CV_kWtrain']))*(combined_stats_sets[sample_set]['CV_kW_ratio'])**0.25
# CVratio data for plotting
CVratio_data_refCurves = CVratio_generator(combined_stats_sets)

CVratio_data_refCurves['psi'] = psi(CVratio_data_refCurves,Twb_max,Twb_min)
CVratio_data_refCurves

**5.12.2 CV barplots: Reference curves method**

In [ ]:
CV_barplots(CVratio_data_refCurves,[0,40],"Reference curves method")

**5.12.3 RMSE barplots: Reference curves method**

In [ ]:
RMSE_barplots(CVratio_data_refCurves,[0,50],"Reference curves method")

**5.12.4 Monthly variation of test data CV and RMSE: Best Samples**

In [ ]:
#Chiller Parameters
Q_rated = 2110 #kW
ELE_rated = 341.4 #kW
COP_rated = Q_rated/ELE_rated
CWRT_rated = (84-32)*5/9
CHWST_rated = (44-32)*5/9

#CANDIDATE CURVES
CAPFT = [[0.5359238,0.01785857,0.001579272,0.04522868,-0.001632172,0.0006008061],
        [0.6322235,0.01553954,-0.00914977,0.05284289,-0.002818835,0.007074096],
        [0.5474968,-0.004800356,-0.006845886,0.06302552,-0.002911528,0.00659869],
        [0.7585741,-0.05127829,0.005848602,0.04235544,-0.001606561,0.0009364594],
        [0.7734286,0.04848829,-0.001859209,0.02611971,-0.001243562,0.0007210181]]

EIRFT = [[0.6270855,-0.04119654,0.001327107,0.02661446,-0.0001856827,0.0005094942],
        [0.910673,-0.00378641,-0.001084261,-0.007148246,0.0005178799,0.00001818623],
        [0.8234137,0.01308264,-0.001147806,-0.004071161,0.0005220869,-0.0006312495],
        [0.7563409,-0.01966623,-0.0003927494,0.02973607,0.000129923,-0.0004325357],
        [0.7203736,0.0103046,-0.0007854237,-0.001206349,0.000821006,-0.001455414]]

EIRFPLR = [[0.2469187,0.186269,0.5660904],[0.2595694,0.1889684,0.551845],[0.2653493,0.144462,0.5911158],
          [0.06917141,0.1743339,0.7524059],[0.08653354,0.1737382,0.7403679]]

best_samples = {'1_month':'sample6','2_months':'sample6','3_months':'sample5','4_months':'sample5'}
stats_refCurves = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
best_models_refCurves = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}

for sample_set,best_sample in best_samples.items():
    model_vars = metering_sample_sets[sample_set][best_sample][['Time_range','CHLR_LCHWT','CWET','Load','ELE']]
    model_vars.columns = ['Time_range','CHWST','CWRT','Load','ELE']
    
    #Units conversion to kW and C
    model_vars['CHWST'] = (model_vars['CHWST']-32)*5/9
    model_vars['CWRT'] = (model_vars['CWRT']-32)*5/9

    curve_vars_dfs = []
    CV_star = []
    Qref_values = []
    Pref_values = []
    for i in range(len(CAPFT)):
        curve_vars = model_vars.copy()
        #Chiller Curve functions
        def capft_curve(CHWST,CWRT,a0=CAPFT[i][0],a1=CAPFT[i][1],a2=CAPFT[i][2],a3=CAPFT[i][3],a4=CAPFT[i][4],a5=CAPFT[i][5]):
            return a0+a1*CHWST+a2*CHWST**2+a3*CWRT+a4*CWRT**2+a5*CHWST*CWRT
        
        def eirft_curve(CHWST,CWRT,b0=EIRFT[i][0],b1=EIRFT[i][1],b2=EIRFT[i][2],b3=EIRFT[i][3],b4=EIRFT[i][4],b5=EIRFT[i][5]):
            return b0+b1*CHWST+b2*CHWST**2+b3*CWRT+b4*CWRT**2+b5*CHWST*CWRT
        
        def eirfplr_curve(PLR,c0=EIRFPLR[i][0],c1=EIRFPLR[i][1],c2=EIRFPLR[i][2]):
            return c0+c1*PLR+c2*PLR**2

        #Curve computations
        curve_vars['CAPFT']=np.vectorize(capft_curve)(curve_vars['CHWST'],curve_vars['CWRT'])
        curve_vars['EIRFT']=np.vectorize(eirft_curve)(curve_vars['CHWST'],curve_vars['CWRT'])
        CAPFT_rated = capft_curve(CHWST_rated,CWRT_rated)
        Qref = Q_rated/CAPFT_rated
        curve_vars['PLR']= curve_vars['Load']/(Qref*curve_vars['CAPFT'])
        curve_vars['EIRFPLR'] = np.vectorize(eirfplr_curve)(curve_vars['PLR'])
        curve_vars['K'] = curve_vars['CAPFT']*curve_vars['EIRFT']*curve_vars['EIRFPLR']/curve_vars['ELE']
        curve_vars['L'] = curve_vars['K']**2
        #Pref = len(curve_vars)/curve_vars['K'].sum()
        Pref = curve_vars['K'].sum()/curve_vars['L'].sum()
        curve_vars['ELE_pred'] = curve_vars['CAPFT']*curve_vars['EIRFT']*curve_vars['EIRFPLR']*Pref
        curve_vars['sq_res_ratio'] = ((curve_vars['ELE']-curve_vars['ELE_pred'])/curve_vars['ELE'])**2 

        curve_vars_dfs.append(curve_vars)
        Qref_values = np.append(Qref_values,Qref)
        Pref_values = np.append(Pref_values,Pref)
        CV_star = np.append(CV_star,np.sqrt(curve_vars['sq_res_ratio'].sum()/len(curve_vars))*100)

    #Choice of the best curve for the chiller
    CV_best = CV_star.min()
    CV_best_posn = CV_star.argmin()
    Qref_best = Qref_values[CV_best_posn]
    Pref_best = Pref_values[CV_best_posn]
    CAPFT_best = CAPFT[CV_best_posn]
    EIRFT_best = EIRFT[CV_best_posn]
    EIRFPLR_best = EIRFPLR[CV_best_posn]
    
    model_vars = curve_vars_dfs[CV_best_posn]
    model_vars['COP'] = model_vars['Load']/model_vars['ELE']
    
    #Selection of the best models to be used in the proceeding section
    model = {'Qref':Qref_best,'Pref':Pref_best,'CAPFT':CAPFT_best,'EIRFT':EIRFT_best,'EIRFPLR':EIRFPLR_best}
    best_models_refCurves[sample_set] = model
    
    #Model power train stats
    model_vars['COP_pred'] = model_vars['Load']/model_vars['ELE_pred']
    model_vars['res^2'] = (model_vars['ELE']-model_vars['ELE_pred'])**2
    model_vars['var'] = (model_vars['ELE']-model_vars['ELE'].mean())**2

    R2 = 1-model_vars['res^2'].sum()/model_vars['var'].sum()
    MAE_kWtrain = mean_absolute_error(model_vars['ELE'],model_vars['ELE_pred'])
    MBE_kWtrain = MAE_kWtrain*100/np.mean(model_vars['ELE'])
    MSE_kWtrain = mean_squared_error(model_vars['ELE'],model_vars['ELE_pred'])
    RMSE_kWtrain = np.sqrt(MSE_kWtrain)
    CV_kWtrain = RMSE_kWtrain*100/np.mean(model_vars['ELE'])

    #Model predictions: Test data with varying months
    test_vars = test_sample_sets[sample_set][best_sample][['Time_range','CHLR_LCHWT','CWET','Load','ELE','month']]
    
    test_vars.columns = ['Time_range','CHWST','CWRT','Load','ELE','month']

    #Units conversion to kW and C
    test_vars['CHWST'] = (test_vars['CHWST']-32)*5/9
    test_vars['CWRT'] = (test_vars['CWRT']-32)*5/9

    test_vars['COP'] = test_vars['Load']/test_vars['ELE']
    test_vars['1/COP'] = 1/test_vars['COP']

    #Model prediction with the test data
    test_vars['CAPFT'] = capft(test_vars['CHWST'],test_vars['CWRT'],CAPFT_best[0],CAPFT_best[1],CAPFT_best[2],
                               CAPFT_best[3],CAPFT_best[4],CAPFT_best[5])
    test_vars['EIRFT'] = eirft(test_vars['CHWST'],test_vars['CWRT'],EIRFT_best[0],EIRFT_best[1],EIRFT_best[2],
                               EIRFT_best[3],EIRFT_best[4],EIRFT_best[5])
    test_vars['PLR'] = test_vars['Load']/(Qref_best*test_vars['CAPFT'])
    test_vars['EIRFPLR'] = eirfplr(test_vars['PLR'],EIRFPLR_best[0],EIRFPLR_best[1],EIRFPLR_best[2])
    #Power prediction stats: Test data
    test_vars['ELE_pred'] = Pref_best*test_vars['CAPFT']*test_vars['EIRFT']*test_vars['EIRFPLR']

    
    # Split the test data into monthly datasets
    test_data_months = {'Jan':[],'Feb':[],'Mar':[],'Apr':[],'May':[],'Jun':[],'Jul':[],'Aug':[],
                       'Sep':[],'Oct':[],'Nov':[],'Dec':[]}
    stats = []
    for month in test_data_months:
        test_data_months[month] = test_vars[test_vars['month']==month]
        test_vars_month = test_data_months[month]

        #Power prediction stats: Monthly Test data
        MAE_kWtest = mean_absolute_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        MBE_kWtest = MAE_kWtest*100/np.mean(test_vars_month['ELE'])
        MSE_kWtest = mean_squared_error(test_vars_month['ELE'],test_vars_month['ELE_pred'])
        RMSE_kWtest = np.sqrt(MSE_kWtest)
        CV_kWtest = RMSE_kWtest*100/np.mean(test_vars_month['ELE'])

        stats = np.append(stats,[MBE_kWtrain,RMSE_kWtrain,CV_kWtrain,MBE_kWtest,RMSE_kWtest,CV_kWtest])
    
    stats_refCurves[sample_set] = pd.DataFrame(stats.reshape((12,6)), index=test_data_months.keys(), 
                                  columns=['MBE_kWtrain','RMSE_kWtrain','CV_kWtrain',
                                           'MBE_kWtest','RMSE_kWtest','CV_kWtest'])
    
    #Creating a new column containing the index values
    stats_refCurves[sample_set] = stats_refCurves[sample_set].reset_index().rename(columns={'index': 'metering period'})

stats_refCurves['4_months']

In [ ]:
selected_sample_CVplots(stats_refCurves,'Reference curves method',[0,30])

In [ ]:
selected_sample_RMSEplots(stats_refCurves,'Reference curves method',[0,50])

In [ ]:
selected_sample_CV_RMSE_plots(stats_refCurves,"Reference curves method",CV_ylim=[0,40],CV_yticks=np.arange(0,40+1,5),
                              RMSE_yticks=np.arange(0,60+1,15),ticks_rotation=-30,CVtext_y=-1.5,RMSEtext_y=1)

**5.12.5 Variation prediction error for the best samples**

In [ ]:
error_data = {'1_month':[],'2_months':[],'3_months':[],'4_months':[]}
for sample_set in error_data.keys():
    error_data[sample_set] = error_computation(chillerdata).refCurves_model(best_models_refCurves[sample_set])
     
# PLOTS
error_variation_plot(error_data,"Reference curves method")

**5.12.6 Error variation: Overall best sample**

In [ ]:
#Best overall sample
best_sample_set = '3_months'

#Error variation of the overall best sample
error_data = error_computation(chillerdata).refCurves_model(best_models_refCurves[best_sample_set])

# PLOTS
error_overall_best(error_data,"Reference curves method",figsize=(10,6))

**5.13 Best metering periods**

In [ ]:
all_data = {'temp_ind':CVratio_data_temp_independent,'quasi_emp':CVratio_data_quasi_empirical,
            'GN_no_interc':CVratio_data_GN_no_inter,'GN_w_interc':CVratio_data_GN_w_inter,'GN_DS':CVratio_data_GN_DS,
            'GN_split':CVratio_data_GN_split_Qleak,'GN_Foliaco':CVratio_data_GN_Foliaco,'biquadratic':CVratio_data_biquadratic,
            'multivariate':CVratio_data_multivariate,'4CP':CVratio_data_4CP,'RefCurves':CVratio_data_refCurves}

for model in all_data.keys():
    best_periods[('1 month','CV_kW_net')][model] = all_data[model]['CV_kW_net'][:12].min()
    best_periods[('1 month','Best metering period')][model] = all_data[model]['metering period'].iloc[
        all_data[model]['CV_kW_net'][:12].idxmin()]

    best_periods[('2 months','CV_kW_net')][model] = all_data[model]['CV_kW_net'][12:24].min()
    best_periods[('2 months','Best metering period')][model] = all_data[model]['metering period'].iloc[
        all_data[model]['CV_kW_net'][12:24].idxmin()]

    best_periods[('3 months','CV_kW_net')][model] = all_data[model]['CV_kW_net'][24:36].min()
    best_periods[('3 months','Best metering period')][model] = all_data[model]['metering period'].iloc[
        all_data[model]['CV_kW_net'][24:36].idxmin()]

    best_periods[('4 months','CV_kW_net')][model] = all_data[model]['CV_kW_net'][36:48].min()
    best_periods[('4 months','Best metering period')][model] = all_data[model]['metering period'].iloc[
        all_data[model]['CV_kW_net'][36:48].idxmin()]
    
#Save data to excel
best_periods.to_excel('BestPeriods.xlsx',sheet_name='Best_Periods')

best_periods

**5.14 Average performances**

In [ ]:
for model in all_data.keys():
    performance_averages[('1 month','Min')][model] = all_data[model]['CV_kW_net'][:12].min()
    performance_averages[('2 months','Min')][model] = all_data[model]['CV_kW_net'][12:24].min()
    performance_averages[('3 months','Min')][model] = all_data[model]['CV_kW_net'][24:36].min()
    performance_averages[('4 months','Min')][model] = all_data[model]['CV_kW_net'][36:48].min()
    
    performance_averages[('1 month','Max')][model] = all_data[model]['CV_kW_net'][:12].max()
    performance_averages[('2 months','Max')][model] = all_data[model]['CV_kW_net'][12:24].max()
    performance_averages[('3 months','Max')][model] = all_data[model]['CV_kW_net'][24:36].max()
    performance_averages[('4 months','Max')][model] = all_data[model]['CV_kW_net'][36:48].max()
    
    performance_averages[('1 month','Average')][model] = all_data[model]['CV_kW_net'][:12].mean()
    performance_averages[('2 months','Average')][model] = all_data[model]['CV_kW_net'][12:24].mean()
    performance_averages[('3 months','Average')][model] = all_data[model]['CV_kW_net'][24:36].mean()
    performance_averages[('4 months','Average')][model] = all_data[model]['CV_kW_net'][36:48].mean()
    
    performance_averages[('1 month','Median')][model] = all_data[model]['CV_kW_net'][:12].median()
    performance_averages[('2 months','Median')][model] = all_data[model]['CV_kW_net'][12:24].median()
    performance_averages[('3 months','Median')][model] = all_data[model]['CV_kW_net'][24:36].median()
    performance_averages[('4 months','Median')][model] = all_data[model]['CV_kW_net'][36:48].median()

performance_averages.to_excel('Overall Performances.xlsx',sheet_name='Overall Summary')
performance_averages

**5.15 Overall Performance Plots**

In [ ]:
key_model_performances = {'GN_no_interc':[],'biquadratic':[],'4CP':[],'RefCurves':[]}
for model in key_model_performances.keys():
    key_model_performances[model] = all_data[model]
    
titles = ['(a)','(b)','(c)','(d)']
labels = ['$GNF_{zero}$','BQM','4CP','RefCur']
markers = ['o','s','^','*']

fig,ax = plt.subplots(2,2, figsize=(10,10),dpi=200)
for i,model in enumerate(key_model_performances):
    #1-month plots
    sns.lineplot(data=key_model_performances[model][key_model_performances[model]['Sample_set']=='1_month'],
                     x='metering period',y='CV_kW_net',ax=ax[0][0],label=labels[i],marker=markers[i])
    #2-month plots
    sns.lineplot(data=key_model_performances[model][key_model_performances[model]['Sample_set']=='2_months'],
                     x='metering period',y='CV_kW_net',ax=ax[0][1],label=labels[i],marker=markers[i])
    #3-month plots
    sns.lineplot(data=key_model_performances[model][key_model_performances[model]['Sample_set']=='3_months'],
                 x='metering period',y='CV_kW_net',ax=ax[1][0],label=labels[i],marker=markers[i])
    #4-month plots
    sns.lineplot(data=key_model_performances[model][key_model_performances[model]['Sample_set']=='4_months'],
                 x='metering period',y='CV_kW_net',ax=ax[1][1],label=labels[i],marker=markers[i]) 
    
#Axes Settings
ax[0][0].set_yscale('log')
ax[0][0].set_ylim(1,30000)  #Limits determined from the model with the worst CV_net
ax[0][0].set_xlabel('Metering Period',fontsize=12)
ax[0][0].set_ylabel('$CV_{net}$', fontsize=12)
ax[0][0].set_xticks(np.arange(1,12,2))
ax[0][0].tick_params(rotation=-30,labelsize=10,axis='x')
ax[0][0].grid(which='major',alpha=0.5)
ax[0][0].set_xlim(0,11)
fig.text(0.275,.463,titles[0],fontweight='bold',fontsize=12)

ax[0][1].set_yscale('log')
ax[0][1].set_ylim(1,3000)
ax[0][1].set_xlabel('Metering Period',fontsize=12)
ax[0][1].set_ylabel('$CV_{net}$', fontsize=12)
ax[0][1].set_xticks(np.arange(1,12,2))
ax[0][1].tick_params(rotation=-30,labelsize=10,axis='x')
ax[0][1].grid(which='major',alpha=0.5)
ax[0][1].set_xlim(0,11)
fig.text(0.7,.463,titles[1],fontweight='bold',fontsize=12)
                 
ax[1][0].set_yscale('log')
ax[1][0].set_ylim(1,200)
ax[1][0].set_xlabel('Metering Period',fontsize=12)
ax[1][0].set_ylabel('$CV_{net}$', fontsize=12)
ax[1][0].set_xticks(np.arange(1,12,2))
ax[1][0].tick_params(rotation=-30,labelsize=10,axis='x')
ax[1][0].grid(which='major',alpha=0.5)
ax[1][0].set_xlim(0,11)
fig.text(0.275,0.02,titles[2],fontweight='bold',fontsize=12)

ax[1][1].set_yscale('log')
ax[1][1].set_ylim(1,200)
ax[1][1].set_xlabel('Metering Period',fontsize=12)
ax[1][1].set_ylabel('$CV_{net}$', fontsize=12)
ax[1][1].set_xticks(np.arange(1,12,2))
ax[1][1].tick_params(rotation=-30,labelsize=10,axis='x')
ax[1][1].grid(which='major',alpha=0.5)
ax[1][1].set_xlim(0,11)
fig.text(0.7,0.02,titles[3],fontweight='bold',fontsize=12)

fig.subplots_adjust(hspace=0.33)
fig.savefig('CV_net Variation with metering periods.png',bbox_inches='tight')

In [ ]:
key_model_performances['GN_no_interc']